##Introduction to Data Analysis Techniques for Cosmic Microwave Background  Maps

                          Jeff McMahon for the 2015 La Serena Data Science Summer School

The Cosmic Microwave Background (CMB) is the oldest observable light in the universe  As such it carries a wealth of cosmological information including: (1) signals from the early universe (primary anisotropy), and (2) distortions imprinted as this light propagates through the universe and encounters collapsed structures (secondary anisotropy .  Measurements of these signals give us important measurements and constraints on inflationary parameters, dark energy, dark matter, the sum of the neutrino masses, and many astrophysical processes.  The development of CMB instruments and analysis techniques is rapidly evolving.

This notebook provides an introduction to analysis techniques for CMB maps as they pertain to high resolution CMB instruments such as the Atacama Cosmology Telescope (ACT) and the South Pole Telescope (SPT).  These state of the art instruments have thousands of detectors (pixels) deployed on 6 and 10 meter telescopes which produce arc minute resolution beams at 150 GHz.  These telescopes observe by scanning across the sky.  The data from each detector are read out at high speed (typically > 50 Hz) to produce 'detector time streams' which are recored along with pointing information and other ancillary data.  Sophisticated codes are used to convert these time streams into maps of the CMB sky.  These maps can have correlated noise that must be accounted for in data analysis.   The mapmaking algorithms are not described here but represent a very interesting big data challenge as they require cleaning time streams by filtering, identifying transient events, and ultimately compressing ~Tb of data down to maps that are typically 100 Mb or less.  

Here are example maps from SPT (top, taken from a phys.org article from April 2, 2012 titled "
South Pole Telescope hones in on dark energy, neutrinos
"
) and ACTPol (bottom, taken from the Naess et al CMB polarization paper).  Many features are obvious in these maps including: (1) the primary CMB visible as waves in the intensity, (2) active galactic nuclei and other bright astrophysical point sources which manifest as bright dots, (3) clusters of galaxies which show up as darkened point sources, and (4) horizontal stripes (in the SPT map) that are the result of the noise and map filtering.  The ACTPol figure shows multiple maps; the T is temperature, Q and U are polarization, and E and B are also polarization but decomposed into a basis such that the E is the amplitude of the curl-free component, and B is the amplitude of the divergence free component of the polarization vector field.


<img src="http://cdn.phys.org/newman/gfx/news/hires/2012/newcosmologi.jpg">

<img src="http://www.classe.cornell.edu/rsrc/Home/NewsAndEvents/CornellExperimentalCosmologyNews20140528/maps_b.png">



While the current instruments (ACTPol and SPTPol) have multiple frequencies and polarization sensitivity, for simplicity we consider only a single frequency (150 GHz) and only temperature.  Adding extra frequency channels or polarization add the cost of slightly more work and would make this presentation more confusing.  Therefore we leave these complications for a future treatment and focus on temperature alone.  This allows us to show the basics of monty carlo analysis of both the angular power spectrum and matched filter techniques for studying Sunyaev-Zeldovich (SZ) effect.

The note book is organized as follows.   We beginning by building simulation of the CMB sky including the primary CMB, point sources (AGN and dusty galaxies), the SZ effect, and (if time) CMB lensing.   To these maps, we fold in instrumental effects including the beam, instrument and atmospheric noise.  We then present several analysis techniques including monty carlo estimation of power spectra and matched filter techniques for extraction of sources    An example of a stacking analysis is presented as an example of a cross-correlation with external data sets.  Cross-correlation is a very active field of research.

In the interest of simplicity we use approximate distributions for source and SZ counts with combinations of exponential and poisson distributions.   We note explicitly where we make these approximations.

## Code preliminaries
We use the following libraries in this code.  All are available through Anaconda.

In [ ]:
## add your filepath for the calibraiton files here, comment out other peoples so it is quick to switch platforms
filepath = ""

## filepath for the ACTMaps, same etiquite as above
ACTMapfilepath = "" ## filepath for the ACTPol maps

## write out simulated maps
write_out_maps = False
N_iterations = 16   ## number of iterations to run in the Monte Carlo sims

import numpy as np
import matplotlib
import sys
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.interpolate import interp1d
import astropy.io.fits as pyfits

%matplotlib inline

## Angular Power Spectrum
The majority of the information content of the CMB is contained in its angular power spectrum.   This spectrum is the amplitude squared of the magnitude of the temperature fluctuations as a function of $\ell$.  Here $\ell$ is the variable you have seen before with the spherical harmonics (e.g., $Y_{\ell m}$).  $\ell = 0$ corresponds to a constant temperature across the sky, $\ell = 200$ corresponds approximately to scales of $1^\circ$.  For a given set of input cosmological parameters these spectra can be computed with codes including CMBFAST or the more modern equivalent CAMB.  The spectrum used below was computed with CAMB web interface on Lambda.

http://lambda.gsfc.nasa.gov/toolbox/tb_camb_form.cfm

It is fun to play with parameters and see how the spectrum changes. 

The relative locations and amplitudes of the peaks carry a great deal of information.  The location of the first peak at $\ell \sim 200$ provided the first definitive measurement that our universe is flat.  The shape of the spectrum constraints a wide variety of cosmological parameters including dark energy and dark matter.  The fall off at high ell (e.g. $\ell \gtrsim 1000$ is called the damping tail and it provides constrains on the spectral index from inflation.  A wealth of cosmological parameters are constrained by measurements of this spectrum.  

At this point the temperature spectrum is well measured and the open frontiers are polarization and secondary anisotropies that are not included in this spectrum.   We will now build simulated skies including the CMB anisotropy and secondary anisotropies.  After that we will show how to estimate the power spectrum from map data using monte carlo techniques.   This monty carlo approach can be used for interpretation of polarization data and for the interpretation of cross-correlations between CMB and other survey data.  Thus this exercise is of great utility.

Here is how to read in an plot the CMB temperature spectrum from a CAMB simulation.

In [ ]:
nbig = 10000  ## zero pad the CMB specra to avoid crashing the code when making maps with small pixels
# read in the input CMB spectra
ell, DlTT,DlEE,DlBB, DlTE= np.loadtxt(filepath + "CMB_fiducial_totalCls.dat", usecols=(0, 1,2,3,4), unpack=True) 

##
ell_big = np.arange(nbig)
DlTT_big = np.zeros(nbig)
DlTT_big[ell.astype(int)] = DlTT
DlEE_big = np.zeros(nbig)
DlEE_big[ell.astype(int)] = DlEE
DlBB_big = np.zeros(nbig)
DlBB_big[ell.astype(int)] = DlBB
DlTE_big = np.zeros(nbig)
DlTE_big[ell.astype(int)] = DlTE

ell = ell_big
DlTT = DlTT_big + 1e-3   ### the 1e-3 factor maps plotting easy
DlEE = DlEE_big + 1e-3
DlBB = DlBB_big + 1e-3
DlTE = DlTE_big

Temp_point_source_spectrum = DlTT[3000]*(ell/3000.)**2.
Pol_point_source_spectrum = DlEE[4500]*(ell/4500.)**2.

DlTT_PS = DlTT + Temp_point_source_spectrum   ### these are used for computing the transer functions
DlEE_PS = DlEE + Pol_point_source_spectrum
DlBB_PS = DlBB + Pol_point_source_spectrum


plt.semilogy(ell,DlTT,'r')
plt.semilogy(ell,DlEE,'g')
plt.semilogy(ell,DlBB,'b')
plt.ylabel('$D^{TT}_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(ell,DlTE)
plt.ylabel('$D^{BB}_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

#White = 3e-4
#one_over_f = 1.
#plt.semilogy(ell,(ell*(ell+1)/np.pi/2.)*(White + (ell/500.)**-4. * one_over_f )  )
#plt.ylabel('$D^{BB}_{\ell}$ [$\mu$K$^2$]')
#plt.xlabel('$\ell$')
#plt.axis([0, 10000, 1e1, 1e5])
#plt.show()



This plot is the angular power spectrum of the CMB temperature anisotropies.


## Temperature Anisotropy Map
In this step we generate a simulated map of the CMB sky with the spectrum we read in above.  Since the power spectrum is a function of $\ell$ we need to do much of the work in harmonic space.  If we were generating a map on the full sky we would need to work with spherical harmonics.  Here we consider a small patch of sky ($\sim 10^\circ \times 10^\circ$) were we cam is the 'flat-sky' approximation and replace $\ell$ with $k = \sqrt{k_x^2 + k_y^2}$.  There is a linear dependance between these variables defined by $\ell = k* 2 \pi$.

In the flat sky approximation we generate a CMB map by:  (1) generating a 2 d power spectrum by revolving the above spectrum (properly normalized) about the axis in polar coordinates, (2) generating a gaussian random map with unit variance, (3) multiplying that maps from 1 and 2, and (4) Fourier transforming this to get a real space map.  We provide a function to do this and a function to plot this (and other maps) with a uniform color scale.

In [ ]:
## variables to set up the size of the map
N = 2**10.  # this is the number of pixels in a linear dimension
            ## since we are using lots of FFTs this should be a factor of 2^N
pix_size  = .5 # size of a pixel in arcminutes

Nsqdeg=(N*pix_size/60.)**2.


## variables to set up the map plots
c_min = -400  # minimum for color bar
c_max = 400   # maximum for color bar
X_width = N*pix_size/60.  # horizontal map width in degrees
Y_width = N*pix_size/60.  # vertical map width in degrees


def make_CMB_maps(N,pix_size,ell,DlTT,DlEE,DlTE,DlBB):
    "makes a realization of a simulated CMB sky map"

    # convert Dl to Cl
    ClTT = DlTT * 2 * np.pi / (ell*(ell+1.))
    ClEE = DlEE * 2 * np.pi / (ell*(ell+1.))
    ClTE = DlTE * 2 * np.pi / (ell*(ell+1.))
    ClBB = DlBB * 2 * np.pi / (ell*(ell+1.))
    
    ClTT[0:1] = 0.
    ClEE[0:1] = 0.
    ClTE[0:1] = 0.
    ClBB[0:1] = 0.

    correlated_part_of_E = ClTE / np.sqrt(ClTT)
    uncorrelated_part_of_EE = ClEE - ClTE**2. / ClTT
    
    correlated_part_of_E[0:1] = 0.
    uncorrelated_part_of_EE[0:1] = 0.
    

    # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) /(N-1.)
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2.)
    ang = np.arctan2(Y,X)
    
    # now make a 2d CMB mask
    ell_scale_factor = 2. * np.pi / (pix_size/60. * np.pi/180.)
    ell2d = R * ell_scale_factor
    ClTT_expanded = np.zeros(ell2d.max()+1)
    ClTT_expanded[0:(ClTT.size)] = ClTT
    ClEE_uncor_expanded = np.zeros(ell2d.max()+1)
    ClEE_uncor_expanded[0:(uncorrelated_part_of_EE.size)] = uncorrelated_part_of_EE
    ClE_corr_expanded = np.zeros(ell2d.max()+1)
    ClE_corr_expanded[0:(correlated_part_of_E.size)] = correlated_part_of_E
    ClBB_expanded = np.zeros(ell2d.max()+1)
    ClBB_expanded[0:(ClBB.size)] = ClBB
    CLTT2d = ClTT_expanded[ell2d.astype(int)]
    ClEE_uncor_2d = ClEE_uncor_expanded[ell2d.astype(int)]
    ClE_corr2d = ClE_corr_expanded[ell2d.astype(int)]
    CLBB2d = ClBB_expanded[ell2d.astype(int)]
    

    
    ## make a plot of the 2d cmb power spectrum, note the x and y axis labels need to be fixed
    #Plot_CMB_Map(CLTT2d**2. *ell2d * (ell2d+1)/2/np.pi,0,np.max(CLTT2d**2. *ell2d * (ell2d+1)/2/np.pi)/10.,ell2d.max(),ell2d.max())  ###
 
    # now make the CMB
    ramdomn_array_for_T = np.fft.fft2(np.random.normal(0,1,(N,N)))
    ramdomn_array_for_E = np.fft.fft2(np.random.normal(0,1,(N,N))) 
    ramdomn_array_for_B = np.fft.fft2(np.random.normal(0,1,(N,N))) 
    FT_2d = np.sqrt(CLTT2d) * ramdomn_array_for_T
    FE_2d = np.sqrt(ClEE_uncor_2d) * ramdomn_array_for_E + ClE_corr2d* ramdomn_array_for_T
    FB_2d = np.sqrt(CLBB2d) * ramdomn_array_for_B
    ## now conver E abd B to Q and U
    FQ_2d = FE_2d* np.cos(2.*ang) - FB_2d * np.sin(2. *ang)
    FU_2d = FE_2d* np.sin(2.*ang) + FB_2d * np.cos(2. *ang)
    
    ## make a plot of the 2d cmb simulated map in fourier space, note the x and y axis labels need to be fixed
    CMB_T = np.fft.ifft2(np.fft.fftshift(FT_2d)) /(pix_size /60.* np.pi/180.)
    CMB_T = np.real(CMB_T)
    CMB_Q = np.fft.ifft2(np.fft.fftshift(FQ_2d)) /(pix_size /60.* np.pi/180.)
    CMB_Q = np.real(CMB_Q)
    CMB_U = np.fft.ifft2(np.fft.fftshift(FU_2d)) /(pix_size /60.* np.pi/180.)
    CMB_U = np.real(CMB_U)

    ###CMB_E = np.fft.ifft2(np.fft.fftshift(FE_2d)) /(pix_size /60.* np.pi/180.)
    ###CMB_E = np.real(CMB_E)
    ###p = Plot_CMB_Map(CMB_E,-40,40,N*pix_size/60.,N*pix_size/60.)
    ###CMB_B = np.fft.ifft2(np.fft.fftshift(FB_2d)) /(pix_size /60.* np.pi/180.)
    ###CMB_B = np.real(CMB_B)
    ###p = Plot_CMB_Map(CMB_B,-40,40,N*pix_size/60.,N*pix_size/60.)
    ## return the map
    return(CMB_T,CMB_Q,CMB_U)
  ###############################

def Plot_CMB_Map(Map_to_Plot,c_min,c_max,X_width,Y_width):
    print("map mean:",np.mean(Map_to_Plot),"map rms:",np.std(Map_to_Plot))
    plt.figure()
    im = plt.imshow(Map_to_Plot, interpolation='bilinear', origin='lower',cmap=cm.RdBu_r)
    im.set_clim(c_min,c_max)
    cbar = plt.colorbar()
    im.set_extent([0,X_width,0,Y_width])
    plt.ylabel('angle $[^\circ]$')
    plt.xlabel('angle $[^\circ]$')
    cbar.set_label('tempearture [uK]', rotation=270)
    plt.show()
    return(0)
  ###############################

def plot_3maps_bokeh(M1,M2,M3):
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, show, output_file, gridplot

    output_notebook()
    TOOLS = 'pan,box_zoom,box_select,crosshair,resize,reset'
    plot_options = dict(width=300, plot_height=300, title=None,tools=TOOLS)


    s1 = figure(x_range=[0, 10], y_range=[0, 10],**plot_options)
    s1.image(image=[M1], x=[0], y=[0], dw=[10], dh=[10], palette="Spectral11")

    # create a new plot and share both ranges
    s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
    s2.image(image=[M2], x=[0], y=[0], dw=[10], dh=[10], palette="Spectral11")
    
    # create a new plot and share only one range
    s3 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
    s3.image(image=[M3], x=[0], y=[0], dw=[10], dh=[10], palette="Spectral11")

    p = gridplot([[s1, s2, s3]])
    show(p)  # open a browser
    return(0)
  ###############################
    
    
## make a CMB T map
CMB_T,CMB_Q,CMB_U = make_CMB_maps(N,pix_size,ell,DlTT,DlEE,DlTE,DlBB)

#####BP = plot_3maps_bokeh(CMB_T,CMB_Q,CMB_U)  ## WARNING_- this plot is nice, but precludes saving
p = Plot_CMB_Map(CMB_T,c_min,c_max,X_width,Y_width)
p = Plot_CMB_Map(CMB_Q,c_min/20.,c_max/20.,X_width,Y_width)
p = Plot_CMB_Map(CMB_U,c_min/20.,c_max/20.,X_width,Y_width)

This contour plot shows simulated CMB map we just generated.  If you generate it multiple times you will find that you get different patterns, however you will see that the typical size of the brightest and darkest spots will stay around $1^\circ$, corresponding to the peak of the angular power spectrum.  All simulated sky maps are displayed with the same color scale to permit visual comparison.

## Point Source Map

Point sources in CMB maps arise from a number of astrophysical objects including Active Galactic Nuclei (AGN), Dust Star Forming Galaxies (DSFGs), and a bright tail of lensed DSFGs.  All of theses objects are interesting in their own right.  For the purposes of our mock skies we approximate these populations as a combination of a faint distribution of sources with a poisson distribution of brightness and small number of very bright sources with and exponentially falling source count.   This approximates the distribution of faint and bright sources seen in real CMB maps.   The source parameters were chosen by eye to look about the same as real maps at 150 GHz.   Detailed source counts have been published and can be consulted to add more reality into these simulations.  Publications also detail the frequency scalings of these sources.  In general DSFGs grow brighter at higher frequencies while AGNs have a spectrum that falls with increasing frequencies.  These behaviors must be included in multifrequency analyses and simulations.

In [ ]:
### paramaters to set up the poisson point sources
Number_of_Sources  = 5000.
Amplitude_of_Sources = 200.
Number_of_Sources_EX = 50.
Amplitude_of_Sources_EX = 1000.

flux_cut = 1e9

### paramaters to set up the sampled source distribution
pol_fraction_mean = 0.01
gethist=np.transpose(np.loadtxt(filepath + "source_distribution_v5.txt"))
S_llim=gethist[:][0]
ns=gethist[:][4]
ns=ns[0:21]
source_density=15.    ## number of sources per square degree
logS_llim = np.log10(S_llim)
bin_width = np.diff(logS_llim)
hist_pdf = ns/(source_density*Nsqdeg)/bin_width
Number_of_Sampled_Sources = np.floor(source_density*Nsqdeg)





def inverse_transform_sampling(hist, bin_edges, n_samples):   
    cum_values = np.zeros(bin_edges.shape)   
    cum_values[1:] = np.cumsum(hist*np.diff(bin_edges)) 
    r = np.random.uniform(0.,np.max(cum_values),n_samples) 
    inv_cdf = interp1d(cum_values, bin_edges)      
    return inv_cdf(r)

def generated_source_component(N,pix_size,Number_of_Sources,pol_fraction_mean,Source_Amplitude_List,flux_cut):
    "makes a realization of a nieve pointsource map"
    PSMap_T = np.zeros([N,N])
    PSMap_Q = np.zeros([N,N])
    PSMap_U = np.zeros([N,N])
    i = 0.
    while (i < Number_of_Sources):
        pix_x = N*np.random.rand() 
        pix_y = N*np.random.rand()
        A_current = Source_Amplitude_List[i]/395.11/200e-9 # amplitude in in uK
        theta = np.random.rand() *2.*np.pi
        if (A_current < flux_cut):
            PSMap_T[pix_x,pix_y] += A_current
            PSMap_Q[pix_x,pix_y] += A_current * pol_fraction_mean * np.sin(2. * theta)
            PSMap_U[pix_x,pix_y] += A_current * pol_fraction_mean * np.cos(2. * theta)
        i = i + 1
    return(PSMap_T,PSMap_Q,PSMap_U)    
  ############################### 
    

def Poisson_source_comonent(N,pix_size,Number_of_Sources,Amplitude_of_Sources):
    "makes a realization of a nieve pointsource map"
    PSMap = np.zeros([N,N])
    i = 0.
    while (i < Number_of_Sources):
        pix_x = N*np.random.rand() 
        pix_y = N*np.random.rand() 
        PSMap[pix_x,pix_y] += np.random.poisson(Amplitude_of_Sources)
        i = i + 1

    return(PSMap)    
  ############################### 

def Exponential_source_comonent(N,pix_size,Number_of_Sources_EX,Amplitude_of_Sources_EX):
    "makes a realization of a nieve pointsource map"
    PSMap = np.zeros([N,N])
    i = 0.
    while (i < Number_of_Sources_EX):
        pix_x = N*np.random.rand() 
        pix_y = N*np.random.rand() 
        PSMap[pix_x,pix_y] += np.random.exponential(Amplitude_of_Sources_EX)
        i = i + 1

    return(PSMap)    
  ############################### 
    
## generate sources
Source_Amplitude_List=10**(inverse_transform_sampling(hist_pdf, logS_llim, Number_of_Sampled_Sources))

## make a point source map
PSMap_T,PSMap_Q,PSMap_U = generated_source_component(N,pix_size,Number_of_Sampled_Sources,pol_fraction_mean,Source_Amplitude_List,flux_cut) 

#PSMap = Poisson_source_comonent(N,pix_size,Number_of_Sources,Amplitude_of_Sources) 
#PSMap +=  Exponential_source_comonent(N,pix_size,Number_of_Sources_EX,Amplitude_of_Sources_EX)

logS = np.log10(Source_Amplitude_List); #S: denotes flux
n_bins = 22
#np.savetxt('Generated_source.txt',Source_Amplitude_List)
print np.max(Source_Amplitude_List)
logS_bin_edges = np.linspace(np.min(logS),np.max(logS),n_bins)
S_bin_edges = np.transpose(10**(logS_bin_edges))

S_bin_gmean = np.zeros(n_bins-1)
S_bin_size = np.zeros(n_bins-1)
for i in range(n_bins-1):
    S_bin_gmean[i] = np.sqrt(S_bin_edges[i] * S_bin_edges[i+1])
    S_bin_size[i] = (S_bin_edges[i+1] - S_bin_edges[i])

num_each_bin, logS_bin_edges1 = np.histogram(logS,logS_bin_edges,density=False)
sr = Nsqdeg/40000.*4.*np.pi 
print num_each_bin
num_each_bin = num_each_bin[::-1]
num_greaterthan_S = np.cumsum(num_each_bin)  #number of sources with flux greater than S
#dNdS = num_greaterthan_S[::-1]/S_bin_size/sr;
print S_bin_gmean
dNdS = num_each_bin[::-1]/S_bin_size/sr;
src_cntg = dNdS * (S_bin_gmean**(5./2.));
plt.loglog(S_bin_gmean,src_cntg,marker='o', linestyle='--')
#xlabel('S (Jy)')
#ylabel('S^{2.5}dN(>S)/dS (Jy^{1.5}sr^{-1}')

#hist,bin_edges = np.histogram(Source_Amplitude_List,bins = 1000,range=[0,Source_Amplitude_List.max()])
#plt.loglog(bin_edges[0:-1],hist*(bin_edges[0:-1])**2.5,'b.')
#plt.xlabel('source amplitude [$\mu$K]')
#plt.ylabel('number or sources')
#plt.axis([1e-3, 10, 1, 1e4])
#plt.show()
    
p=Plot_CMB_Map(PSMap_T,c_min,c_max,X_width,Y_width) 
p=Plot_CMB_Map(PSMap_Q,c_min/100.,c_max/100.,X_width,Y_width) 
p=Plot_CMB_Map(PSMap_U,c_min/100.,c_max/100.,X_width,Y_width) 

The top plot shows a distribution of S**(2.5)dN/dS versus flux S. The lower plot shows a map of the point source map we have simulated.


## SZ Map

Clusters of galaxies imprint a subtle distortion into CMB maps that is most apparent on arc minute scales.    While clusters of galaxies are named after the galaxies bound within them, the galaxies represent only a small fraction of the matter contained within a cluster.   Roughly 80% of the baryons are not contained within galaxies, but rather exist as a cloud of gas bound within the gravitational potential well created by a dark matter halo that caries the vast majority of the mass of the cluster.  Within this well, the dilute gas becomes ionized and heated to temperatures of millions of Kelvin.  Occasionally a CMB photon interacts with one of the hot electrons in this ionized gas.  This interaction (inverse Compton scattering) gives the CMB photon a boost in energy.  Detailed calculations show that this effect (the Sunyev-Zeldovich or SZ effect) leads to decrement of power at frequencies below the 'null' at 220 GHz and extra power at higher frequencies.  This result is redshift independent.  Thus the SZ effect provides a clean way to detect clusters of galaxies and the signal which traces the electron density within the cluster.  The SZ signal is a reasonably good tracer of cluster mass.

For these simulations we treat each cluster as having a "beta profile" and fix each cluster to have an identical angular size.  We draw the distribution of central temperatures from the exponential distribution to simplify the code and reduce the dependance on external libraries.  For more accurate simulations, a range of clusters sizes should be used, a distribution of cluster shapes (with more accurate profiles) should be considered, and the number of clusters as a function of mass and redshift should be chosen to match measurements of the cluster mass function.

In [ ]:
### paramaters to set up the SZ point sources
Number_of_SZ_Clusters  = np.floor(50. * (Nsqdeg/72.))
Mean_Amplitude_of_SZ_Clusters = 100.
SZ_beta = 0.86
SZ_Theta_core = 1.0

def SZ_source_comonent(N,pix_size,Number_of_SZ_Clusters,Mean_Amplitude_of_SZ_Clusters,SZ_beta,SZ_Theta_core,do_plots):
    "makes a realization of a nieve SZ map"
    SZMap = np.zeros([N,N])
    SZcat = np.zeros([3,Number_of_SZ_Clusters]) ## catalogue of SZ sources, X, Y, amplitude
    # make a distribution of point sources with varying amplitude
    i = 0.
    while (i < Number_of_SZ_Clusters):
        pix_x = N*np.random.rand() 
        pix_y = N*np.random.rand() 
        pix_amplitude = np.random.exponential(Mean_Amplitude_of_SZ_Clusters)*(-1.)
        SZcat[0,i] = pix_x
        SZcat[1,i] = pix_y
        SZcat[2,i] = pix_amplitude
        SZMap[pix_x,pix_y] += pix_amplitude
        i = i + 1
    if (do_plots):
        hist,bin_edges = np.histogram(SZMap,bins = 50,range=[SZMap.min(),-10])
        plt.semilogy(bin_edges[0:-1],hist)
        plt.xlabel('source amplitude [$\mu$K]')
        plt.ylabel('number or pixels')
        plt.show()
    
    # make a beta function
    beta = beta_function(N,pix_size,SZ_beta,SZ_Theta_core)
    
    # convovle the beta funciton with the point source amplitude to get the SZ map
    FT_beta = np.fft.fft2(np.fft.fftshift(beta))
    FT_SZMap = np.fft.fft2(np.fft.fftshift(SZMap))
    SZMap = np.fft.fftshift(np.real(np.fft.ifft2(FT_beta*FT_SZMap)))
    
    # return the SZ map
    return(SZMap,SZcat)    
  ############################### 

def beta_function(N,pix_size,SZ_beta,SZ_Theta_core):
  # make a beta function
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) * pix_size
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2.)
    
    beta = (1 + (R/SZ_Theta_core)**2.)**((1-3.*SZ_beta)/2.)

    # return the beta function map
    return(beta)
  ############################### 
    
## make a point source map

SZMap,SZCat = SZ_source_comonent(N,pix_size,Number_of_SZ_Clusters,Mean_Amplitude_of_SZ_Clusters,SZ_beta,SZ_Theta_core,True)

  
p=Plot_CMB_Map(SZMap,c_min,c_max,X_width,Y_width)

The top plot shows the a histogram of the SZ-decrements from our simulated SZ cluster map.  The bottom plot shows our simulated SZ map.


## Sky Map

The sky map is a combination of the CMB anisotropy, a point source map, and an SZ map.  In an appendix we add the impact of CMB lensing.

In [ ]:
T_total_map = CMB_T + PSMap_T + SZMap
Q_total_map = CMB_Q + PSMap_Q
U_total_map = CMB_U + PSMap_U

p=Plot_CMB_Map(T_total_map,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(Q_total_map,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(U_total_map,c_min/20.,c_max/20.,X_width,Y_width)

This plot shows our simulated map that includes CMB, point source, and SZ cluster signals.  Note that the sources seem brighter than what we saw in the real observed maps.   This is not a mistake, as will be seen after we fold in the beam (point spread function) of the instrument.

## Sky Map Convolved with a Beam

Telescopes suffer from diffraction which leads to finite resolution effects.  To account for this we generate a gaussian beam pattern and convolve the map with it.

In [ ]:
beam_size_fwhp = 1.25
#ACTPol_beam_file = filepath + "beam_profile_151107_v0_2014_pa1_jitter_deep56.txt"

def convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,Map):
    "convolves a map with a gaussian beam pattern.  NOTE: pix_size and beam_size_fwhp need to be in the same units" 
    # make a 2d gaussian 
    gaussian = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
  
    # do the convolution
    FT_gaussian = np.fft.fft2(np.fft.fftshift(gaussian))
    FT_Map = np.fft.fft2(np.fft.fftshift(Map))
    convolved_map = np.fft.fftshift(np.real(np.fft.ifft2(FT_gaussian*FT_Map)))
    
    # return the convolved map
    return(convolved_map)
  ###############################   

def make_2d_gaussian_beam(N,pix_size,beam_size_fwhp):
     # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) * pix_size
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2.)
  
    # make a 2d gaussian 
    beam_sigma = beam_size_fwhp / np.sqrt(8.*np.log(2))
    gaussian = np.exp(-.5 *(R/beam_sigma)**2.)
    gaussian = gaussian / np.sum(gaussian)
 
    # return the gaussian
    return(gaussian)
  ###############################     
    
    
## convolvoe the signal part of the map

CMB_T_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,T_total_map)
CMB_Q_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,Q_total_map)
CMB_U_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,U_total_map)

p=Plot_CMB_Map(CMB_T_convolved,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(CMB_Q_convolved,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(CMB_U_convolved,c_min/20.,c_max/20.,X_width,Y_width)

This plot shows the simulated map that includes the CMB, point source, and SZ signals and is convolved with an instrumental beam.  The agreement with the real sky maps is now reasonable, but does not yet include instrumental noise.


## Noise Map

Ground based CMB instruments suffer from several types of noise.  This includes: (1) White noise drawn from a gaussian distribution; (2) atmospheric noise which grows larger on large angular scales, this can be understood in terms of Kolmagrove turbulence; and (3) $1/f$ noise in the detectors.  Both atmospheric and detector $1/f$ manifest as correlated noise in maps space.  However, like the CMB these noise components are very nearly uncorrelated in Fourier space.  We will build these noise terms in Fourier space and display them in map space. 

We choose a white noise level of 10 $\mu$K-arcmin to approximate the current deep maps coming from ACTPol and SPTPol.  The $1/f$ noise is similar to what would be seen in an experiment at the south pole.  When turned on, this noise leads to the effect of "striping" in the maps.  We default to turning this component off since if it is included we would need to filter the maps before estimating the power spectrum.  (This is easy, but is not yet implemented).  The atmospheric noise (Kolmagrove turbulence) has a 2d spectrum similar to CMB leading to similar patterns in the maps.

In [ ]:
white_noise_level = .25
noise_amplitude_1 = 5.
noise_index_1 = .06
noise_amplitude_2 = 2.5
noise_index_2 = .042
white_noise_level_pol = white_noise_level * np.sqrt(2.)
noise_amplitude_1_pol = 0.01
noise_amplitude_2_pol = 0.5 

    
def make_noise_map(N,pix_size,white_noise_level,noise_amplitude_1,noise_index_1,noise_amplitude_2,noise_index_2):
    "makes a realization of instrument noise, atnospher and 1/f noise level set at 1 degrees"
    ## make a white noise map
    white_noise = np.random.normal(0,1,(N,N)) * white_noise_level/pix_size
 
    ## make an atnosperhic noise map
    atnospheric_noise = 0.
    if (noise_amplitude_1 != 0):
        ones = np.ones(N)
        inds  = (np.arange(N)+.5 - N/2.) 
        X = np.outer(ones,inds)
        Y = np.transpose(X)
        R = np.sqrt(X**2. + Y**2.) * pix_size /60. ## angles realative to 1 degrees  
        mag_k = 2 * np.pi/(R+.01)  ## 0.01 is a regularizaiton factor
        noise_map_1 = np.fft.fft2(np.random.normal(0,1,(N,N)))
        noise_map_1  = np.fft.ifft2(noise_map_1 * np.fft.fftshift(mag_k**noise_index_1))* noise_amplitude_1/pix_size

    ## make a 1/f map, along a single direction to illustrate striping 
    oneoverf_noise = 0.
    if (noise_amplitude_2 != 0): 
        ones = np.ones(N)
        inds  = (np.arange(N)+.5 - N/2.) 
        X = np.outer(ones,inds)
        Y = np.transpose(X)
        R = np.sqrt(X**2. + Y**2.) * pix_size /60. ## angles realative to 1 degrees  
        mag_k = 2 * np.pi/(R+.01)  ## 0.01 is a regularizaiton factor
        noise_map_2 = np.fft.fft2(np.random.normal(0,1,(N,N)))
        noise_map_2  = np.fft.ifft2(noise_map_2 * np.fft.fftshift(mag_k**noise_index_2))* noise_amplitude_1/pix_size


    ## return the noise map
    noise_map = np.real(white_noise + noise_map_1 + noise_map_2)
    return(noise_map)
  ###############################

## make an instrument noise map


Noise_T = make_noise_map(N,pix_size,white_noise_level,noise_amplitude_1,noise_index_1,noise_amplitude_2,noise_index_2)
Noise_Q = make_noise_map(N,pix_size,white_noise_level_pol,noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
Noise_U = make_noise_map(N,pix_size,white_noise_level_pol,noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)

p=Plot_CMB_Map(Noise_T,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(Noise_Q,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(Noise_U,c_min,c_max,X_width,Y_width)

This plot shows a realization of instrumental + atmospheric noise.  The atmospheric noise looks similar to the CMB since it has a similar 2d spectrum.    

##Complete Simulated CMB Map

We complete our simulated CMB map by adding the simulated sky map convolved with the beam to our simulated noise map.

In [ ]:
T_total_map_plus_noise = CMB_T_convolved + Noise_T
Q_total_map_plus_noise = CMB_Q_convolved + Noise_Q
U_total_map_plus_noise = CMB_U_convolved + Noise_U

p=Plot_CMB_Map(T_total_map_plus_noise,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(Q_total_map_plus_noise,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(U_total_map_plus_noise,c_min/20.,c_max/20.,X_width,Y_width)

This plot shows a complete simulated CMB map including the key astrophysical and instrumental effects.

# Map Analysis

We will analyze these maps in several ways.  First we will use a monty carlo technique to recover the power spectrum.   Second we will use a matched filter to isolate the SZ signal and point sources.  Third we will illustrate stacking a map on a cluster catalogue from a simulated external (e.g., optical or x-ray) survey.

## power spectrum
 
In this section we compute the power spectrum of a CMB map.  We will work in the flat sky approximation which is described above in the section where we generated the simulated CMB map.  

### apodize  the map to eliminate edge effects

Before taking a 2d FFT (eg the obvious thing to do for computing a power spectrum) we must apodize the maps to eliminate edges effects.  Edge effects come about because the Fourier transform treats a square array as having periodic boundaries.  Thus if we take the Fourier transform of a 2-dimensional map and the values on the left and right side (and also, top and bottom) of the map don't match, we end up generating spurious signals.   In this example we use a cosine window to smoothly roll off the signal to zero as we approach the edges of the map.  There are many choices of windows that trade sensitivity loss, coupling of adjacent modes, and ringing.

In [ ]:
def cosine_window(N):
    "makes a cosine window for apodizing to avoid edges effects in the 2d FFT" 
    # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.)/N *np.pi ## eg runs from -pi/2 to pi/2
    X = np.outer(ones,inds)
    Y = np.transpose(X)
  
    # make a window map
    window_map = (np.cos(X) * np.cos(Y))**.5
   
    # return the window map
    return(window_map)
  ###############################
    
window = (cosine_window(N))
    
T_appodized_map = window * T_total_map_plus_noise
Q_appodized_map = window * Q_total_map_plus_noise
U_appodized_map = window * U_total_map_plus_noise

p=Plot_CMB_Map(T_appodized_map,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(Q_appodized_map,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(U_appodized_map,c_min/20.,c_max/20.,X_width,Y_width)

### convert Q and U maps to E and B maps using Kendrick's no leak estimators

In [ ]:
def kendric_method_precompute_window_derivitives(win,pix_size):
    delta = pix_size * np.pi /180. /60.
    dwin_dx =    ((-1.) * np.roll(win,-2,axis =1)      +8. * np.roll(win,-1,axis =1)     - 8. *np.roll(win,1,axis =1)      +np.roll(win,2,axis =1) ) / (12. *delta)
    dwin_dy =    ((-1.) * np.roll(win,-2,axis =0)      +8. * np.roll(win,-1,axis =0)     - 8. *np.roll(win,1,axis =0)      +np.roll(win,2,axis =0) ) / (12. *delta)
    d2win_dx2 =  ((-1.) * np.roll(dwin_dx,-2,axis =1)  +8. * np.roll(dwin_dx,-1,axis =1) - 8. *np.roll(dwin_dx,1,axis =1)  +np.roll(dwin_dx,2,axis =1) ) / (12. *delta)
    d2win_dy2 =  ((-1.) * np.roll(dwin_dy,-2,axis =0)  +8. * np.roll(dwin_dy,-1,axis =0) - 8. *np.roll(dwin_dy,1,axis =0)  +np.roll(dwin_dy,2,axis =0) ) / (12. *delta)
    d2win_dxdy = ((-1.) * np.roll(dwin_dy,-2,axis =1)  +8. * np.roll(dwin_dy,-1,axis =1) - 8. *np.roll(dwin_dy,1,axis =1)  +np.roll(dwin_dy,2,axis =1) ) / (12. *delta)
    return(dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)

def kendrick_method_TQU_to_fourier_TEB(N,pix_size,Tmap,Qmap,Umap,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy):
    ### the obvious FFTs
    fft_TxW = np.fft.fftshift(np.fft.fft2(Tmap * window))
    fft_QxW = np.fft.fftshift(np.fft.fft2(Qmap * window))
    fft_UxW = np.fft.fftshift(np.fft.fft2(Umap * window))

    ### the less obvious FFTs that go into the no-leak estiamte
    fft_QxdW_dx = np.fft.fftshift(np.fft.fft2(Qmap * dwin_dx))
    fft_QxdW_dy = np.fft.fftshift(np.fft.fft2(Qmap * dwin_dy))
    fft_UxdW_dx = np.fft.fftshift(np.fft.fft2(Umap * dwin_dx))
    fft_UxdW_dy = np.fft.fftshift(np.fft.fft2(Umap * dwin_dy))
    fft_QU_HOT  = np.fft.fftshift(np.fft.fft2( (2. * Qmap * d2win_dxdy) + Umap * (d2win_dy2 - d2win_dx2) ))
    
    ### generate the radial coordinates needed to cary out the conversion
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) /(N-1.)
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2. + 1e-9)  ## the small offset regularizes the 1/ell factors below
    ang =  np.arctan2(Y,X)
    ell_scale_factor = 2. * np.pi / (pix_size/60. * np.pi/180.)
    ell2d = R * ell_scale_factor
        
    #p=Plot_CMB_Map(np.real( ang),-np.pi,np.pi,N,N)
    
    
    ### now compute the estimator
    fTmap = fft_TxW
    fEmap = fft_QxW * np.cos(2. * ang) + fft_UxW * np.sin(2. * ang)
    fBmap = ( fft_QxW * (-1. *np.sin(2. * ang)) + fft_UxW * np.cos(2. * ang))  ## this line is the nominal B estimator
    fBmap = fBmap - complex(0,2.) / ell2d * (fft_QxdW_dx * np.sin(ang) + fft_QxdW_dy * np.cos(ang))
    fBmap = fBmap - complex(0,2.) / ell2d * (fft_UxdW_dy * np.sin(ang) - fft_UxdW_dx * np.cos(ang))
    fBmap = fBmap +  ell2d**(-2.) * fft_QU_HOT

    ### return the complex fourier maps in 2d
    return(fTmap,fEmap,fBmap)

    
    
dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy = kendric_method_precompute_window_derivitives(window,pix_size)

### plots of the window function derivatives
#p=Plot_CMB_Map(dwin_dx,-10,10,X_width,Y_width)
#p=Plot_CMB_Map(dwin_dy,-10,10,X_width,Y_width)
#p=Plot_CMB_Map(d2win_dx2,-10,10,X_width,Y_width)
#p=Plot_CMB_Map(d2win_dy2,-10,10,X_width,Y_width)
#p=Plot_CMB_Map(d2win_dxdy,-10,10,X_width,Y_width)

fTmap,fEmap,fBmap = kendrick_method_TQU_to_fourier_TEB(N,pix_size,T_total_map_plus_noise,Q_total_map_plus_noise,U_total_map_plus_noise,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)

p=Plot_CMB_Map(np.real( (np.fft.ifft2(np.fft.fftshift(fTmap)))),c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(np.real( (np.fft.ifft2(np.fft.fftshift(fEmap)))),c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(np.real( (np.fft.ifft2(np.fft.fftshift(fBmap)))),c_min/200.,c_max/200.,X_width,Y_width)


This shows our simulated map with a cosine window applied to eliminate edge effects.  It is obvious from this map that we are suppressing the signal here.

### Naive Powerspectrum

Here we compute a naive power spectrum and compare it to the input power spectrum for our simulations.  The power spectrum is compute by: (1) applying a 2d FFT, (2) taking the absolute value squared of this map in Fourier space ($k_x$ and $k_y$), and (3) averaging the signal in annular bins of $k = \sqrt{k_x^2 + k_y^2}$.  These bins are converted to bins in $\ell$ with the scaling: $\ell = k* 2 \pi$ per the flat sky approximation.   NOTE: step 3 (averaging in radial bins) is how we convert our 2d Fourier map into a 1d power spectrum.

In [ ]:
#### parameters for setting up the spectrum
delta_ell = 50.
ell_max = 10000.

def calculate_2d_spectrum(FMap1,FMap2,delta_ell,ell_max,pix_size,N):
    "calcualtes the power spectrum of a 2d map by FFTing, squaring, and azimuthally averaging"
    
    # make a 2d ell coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) /(N-1.)
    kX = np.outer(ones,inds) / (pix_size/60. * np.pi/180.)
    kY = np.transpose(kX)
    K = np.sqrt(kX**2. + kY**2.)
    ell_scale_factor = 2. * np.pi 
    ell2d = K * ell_scale_factor
    
    # make an array to hold the power spectrum results
    N_bins = int(ell_max/delta_ell)
    ell_array = np.arange(N_bins)
    CL_array = np.zeros(N_bins)
    
    # get the 2d fourier transform of the map
    #FMap = np.fft.ifft2(np.fft.fftshift(Map))
    PSMap = (np.real(np.conj(FMap1) * FMap2))
    # fill out the spectra
    i = 0
    while (i < N_bins):
        ell_array[i] = (i + 0.5) * delta_ell
        inds_in_bin = ((ell2d >= (i* delta_ell)) * (ell2d < ((i+1)* delta_ell))).nonzero()
        CL_array[i] = np.mean(PSMap[inds_in_bin])
        i = i + 1
 
    # return the power spectrum and ell bins
    return(ell_array,CL_array*np.sqrt(pix_size /60.* np.pi/180.)*2.*1e-12)

## make a power spectrum
fTmap,fEmap,fBmap = kendrick_method_TQU_to_fourier_TEB(N,pix_size,T_total_map_plus_noise,Q_total_map_plus_noise,U_total_map_plus_noise,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)
binned_ell, fiducial_binned_TT_spectrum = calculate_2d_spectrum(fTmap,fTmap,delta_ell,ell_max,pix_size,N)
binned_ell, fiducial_binned_EE_spectrum = calculate_2d_spectrum(fEmap,fEmap,delta_ell,ell_max,pix_size,N)
binned_ell, fiducial_binned_BB_spectrum = calculate_2d_spectrum(fBmap,fBmap,delta_ell,ell_max,pix_size,N)
binned_ell, fiducial_binned_TE_spectrum = calculate_2d_spectrum(fTmap,fEmap,delta_ell,ell_max,pix_size,N)
binned_ell, fiducial_binned_TB_spectrum = calculate_2d_spectrum(fTmap,fBmap,delta_ell,ell_max,pix_size,N)
binned_ell, fiducial_binned_EB_spectrum = calculate_2d_spectrum(fEmap,fBmap,delta_ell,ell_max,pix_size,N)



plt.semilogy(binned_ell,fiducial_binned_TT_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(binned_ell,fiducial_binned_EE_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(binned_ell,fiducial_binned_BB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(ell,DlTT)
plt.semilogy(ell,DlEE)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,fiducial_binned_TE_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(ell,DlTE)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,fiducial_binned_TB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(binned_ell,fiducial_binned_EB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

This plot shows the input CMB power spectrum (green) and the naive power spectrum we estimated from our CMB map (blue).  The naive spectrum does not match the input due to a combination of: instrumental noise, SZ and point source signals; and suppression from the beam and the apodization.   


### Correcting the biases in the naive power spectrum

To correct the naive spectrum and obtain an unbiased estimate of the underlying power spectrum in our simulated map we must correct additive bias and multiplicative bias.   We can relate our measured spectrum $\hat D_\ell$ to the true underlying spectrum $D_\ell$ as follows:

$$\hat D_\ell = T*D_\ell + N. $$

Here $N$ represents an additive noise term and $T$ represents a transfer function of the instrument (beam) and filtering (in this case the apodization, but other processing can enter, for example a 1-d high pass filter to eliminate striping).   All of these terms are functions of $\ell$.

The true power spectrum can be recovered by using monte carlo technqies.  To apply this technique we use simulations to calibrate $T$ and $N$ and then use algebra to correct the naive measurement.  This is compactly expressed in the following equation:

$$ D_\ell = \hat (D_\ell - N)/T. $$

In the next two sections we use simulations to calibrate $T and $N$ to recover an unbiased estimate of the underlying power spectrum.  Subsequently we will used monte carlo simulations to estimate the error bars on this measurement.

### Calibrating the transfer function

The transfer function can be calibrated by: (1) generating sky simulations with a known power spectrum, modeling the transfer function of the instrument and the post-processing, and keeping the noise level to zero,; and (2) calculating the naive power spectrum from each simulation, (3) running many simulations to reduce numerical noise, and (4) dividing the true spectrum by the the average signal only spectrum to recover our estimate for the transfer function.   The accuracy depends on the number of realizations used.  Here  we used 64 realizations as a compromise between speed and accuracy.  

Here we use a CMB only spectrum to estimate the transfer function.  We could improve the estimate of the transfer function by using an input power spectrum that is modified to follow the effect of point sources.  This could be added as an exercise.  In this exercise compare the transfer function to find out how much the choice of input spectrum matters.

In [ ]:
N_spectra_types = 6 ##  TT EE BB TE TB EB

signal_only  = np.zeros([N_spectra_types,N_iterations,int(ell_max/delta_ell)])
i = 0
while (i <N_iterations):
    CMB_T,CMB_Q,CMB_U = make_CMB_maps(N,pix_size,ell,DlTT_PS,DlEE_PS,DlTE,DlBB_PS)
    ## convlove the maps
    CMB_T_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_T)
    CMB_Q_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_Q)
    CMB_U_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_U)
    ## calcuate the power spectra
    fTmap,fEmap,fBmap = kendrick_method_TQU_to_fourier_TEB(N,pix_size,CMB_T_convolved,CMB_Q_convolved,CMB_U_convolved,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)
    binned_ell, binned_TT_spectrum = calculate_2d_spectrum(fTmap,fTmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_EE_spectrum = calculate_2d_spectrum(fEmap,fEmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_BB_spectrum = calculate_2d_spectrum(fBmap,fBmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_TE_spectrum = calculate_2d_spectrum(fTmap,fEmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_TB_spectrum = calculate_2d_spectrum(fTmap,fBmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_EB_spectrum = calculate_2d_spectrum(fEmap,fBmap,delta_ell,ell_max,pix_size,N)
    signal_only[0,i,:] = binned_TT_spectrum
    signal_only[1,i,:] = binned_EE_spectrum
    signal_only[2,i,:] = binned_BB_spectrum
    signal_only[3,i,:] = binned_TE_spectrum
    signal_only[4,i,:] = binned_TB_spectrum
    signal_only[5,i,:] = binned_EB_spectrum
    sys.stdout.write("\r signal only sims, iterations complete: %d of %d" % ((i+1),N_iterations) )
    sys.stdout.flush()
    i = i + 1

In [ ]:
def average_N_spectra(spectra,N_spectra,N_ells):
    N_spectra_types = 6 ##  TT EE BB TE TB EB
    avgSpectra = np.zeros([N_spectra_types,N_ells])
    rmsSpectra = np.zeros([N_spectra_types,N_ells])
    
    # calcuate the average spectrum
    i = 0
    while (i < N_spectra):
        j = 0
        while (j < N_spectra_types):
            avgSpectra[j,:] = avgSpectra[j,:] + spectra[j,i,:]
            j = j + 1
        i = i + 1
    avgSpectra = avgSpectra/(1. * N_spectra)
    
    #calculate the rms of the spectrum
    i =0
    while (i < N_spectra):
        j = 0
        while (j < N_spectra_types):
            rmsSpectra[j,:] = rmsSpectra[j,:] +  (spectra[j,i,:] - avgSpectra[j,:])**2
            j = j + 1
        i = i + 1
    rmsSpectra = np.sqrt(rmsSpectra/(1. * N_spectra))
    
    return(avgSpectra,rmsSpectra)


sig_only_mean_spectrum, rms_not_needed = average_N_spectra(signal_only,N_iterations,int(ell_max/delta_ell))


sub_sampled_TT_CLs = DlTT_PS[binned_ell] * 2. * np.pi / (binned_ell * (binned_ell+1.))
sub_sampled_EE_CLs = DlEE_PS[binned_ell] * 2. * np.pi / (binned_ell * (binned_ell+1.))
sub_sampled_BB_CLs = DlBB_PS[binned_ell] * 2. * np.pi / (binned_ell * (binned_ell+1.))
sub_sampled_TE_CLs = DlTE[binned_ell] * 2. * np.pi / (binned_ell * (binned_ell+1.))


Multiplicative_Bias_est_TT =  sub_sampled_TT_CLs / sig_only_mean_spectrum[0,:]
Multiplicative_Bias_est_EE =  sub_sampled_EE_CLs / sig_only_mean_spectrum[1,:]
Multiplicative_Bias_est_BB =  sub_sampled_BB_CLs / sig_only_mean_spectrum[2,:]
   
## make some plots
#plt.semilogy(binned_ell,binned_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi,color='b')
#plt.semilogy(binned_ell,(sig_only_mean_spectrum)* binned_ell * (binned_ell+1.)/2. / np.pi,color='g')
plt.semilogy(binned_ell,(Multiplicative_Bias_est_TT),color='r')
plt.semilogy(binned_ell,(Multiplicative_Bias_est_EE),color='g')
plt.semilogy(binned_ell,(Multiplicative_Bias_est_BB),color='b')
plt.semilogy(binned_ell,(fiducial_binned_TT_spectrum)*Multiplicative_Bias_est_TT* binned_ell * (binned_ell+1.)/2. / np.pi,'r.')
plt.semilogy(binned_ell,(fiducial_binned_EE_spectrum)*Multiplicative_Bias_est_EE* binned_ell * (binned_ell+1.)/2. / np.pi,'g.')
plt.semilogy(binned_ell,(fiducial_binned_BB_spectrum)*Multiplicative_Bias_est_BB* binned_ell * (binned_ell+1.)/2. / np.pi,'b.')
plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.semilogy(ell,DlBB,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,(fiducial_binned_TE_spectrum)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE)* binned_ell * (binned_ell+1.)/2. / np.pi,'r.')
plt.plot(ell,DlTE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,(fiducial_binned_TB_spectrum)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'g.')
plt.plot(binned_ell,(fiducial_binned_EB_spectrum)*np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'b.')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

This plot shows the estimate of the CMB power spectrum after correcting for the multiplicative bias (transfer function) in yellow.   In addition we show (red) the input CMB power spectrum, (green) the average of the signal only simulations, (blue, lower) the transfer function, and (blue, upper) the naive power spectrum of our map.  Consider how all these curves relate to creating the yellow estimate.


### calibrating the noise bias

The noise bias can be computed by running noise only simulations through the naive power spectrum estimator and computing the average power spectrum.

NOTE: An alternative approach exists for dealing with the noise.  If you can subdivide your data into subsets with common signal but independent noise, one can compute "cross-spectra" between these subsets.  (You compute these by doing a 2d FFT on each subset and then multiplying one by the complex conjugate of the other.)   This results in some information loss (since you are throwing out the auto-correlation of each subset, but it completely eliminates potential measurement bias from an incorrect noise model.  

As a two part exercise: (1) use the wrong noise model in analyzing the power spectrum to see what happens, and (2) implement a cross-spectrum estimator to see that this noise bias goes away with cross spectra.  Also note that the error bars grow with the cross spectrum. 

In [ ]:
noise_only  = np.zeros([N_spectra_types,N_iterations,int(ell_max/delta_ell)])
i = 0
while (i <N_iterations):
    Noise_T = make_noise_map(N,pix_size,white_noise_level,noise_amplitude_1,noise_index_1,noise_amplitude_2,noise_index_2)
    Noise_Q = make_noise_map(N,pix_size,white_noise_level_pol,noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
    Noise_U = make_noise_map(N,pix_size,white_noise_level_pol,noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
    fTmap,fEmap,fBmap = kendrick_method_TQU_to_fourier_TEB(N,pix_size,Noise_T,Noise_Q,Noise_U,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)
    binned_ell, binned_TT_spectrum = calculate_2d_spectrum(fTmap,fTmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_EE_spectrum = calculate_2d_spectrum(fEmap,fEmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_BB_spectrum = calculate_2d_spectrum(fBmap,fBmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_TE_spectrum = calculate_2d_spectrum(fTmap,fEmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_TB_spectrum = calculate_2d_spectrum(fTmap,fBmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_EB_spectrum = calculate_2d_spectrum(fEmap,fBmap,delta_ell,ell_max,pix_size,N)
    noise_only[0,i,:] = binned_TT_spectrum
    noise_only[1,i,:] = binned_EE_spectrum
    noise_only[2,i,:] = binned_BB_spectrum
    noise_only[3,i,:] = binned_TE_spectrum
    noise_only[4,i,:] = binned_TB_spectrum
    noise_only[5,i,:] = binned_EB_spectrum
    sys.stdout.write("\r noise only sims, iterations complete: %d of %d" % ((i+1),N_iterations) )
    sys.stdout.flush()
    i = i + 1

In [ ]:
noise_only_mean_spectrum, rms_not_needed = average_N_spectra(noise_only,N_iterations,int(ell_max/delta_ell))

Additive_Bias_TT = noise_only_mean_spectrum[0,:]
Additive_Bias_EE = noise_only_mean_spectrum[1,:]
Additive_Bias_BB = noise_only_mean_spectrum[2,:]
Additive_Bias_TE = noise_only_mean_spectrum[3,:]
Additive_Bias_TB = noise_only_mean_spectrum[4,:]
Additive_Bias_EB = noise_only_mean_spectrum[5,:]



plt.semilogy(binned_ell,(Additive_Bias_TT)* binned_ell * (binned_ell+1.)/2. / np.pi,'r.')
plt.semilogy(binned_ell,(Additive_Bias_EE)* binned_ell * (binned_ell+1.)/2. / np.pi,'g.')
plt.semilogy(binned_ell,(Additive_Bias_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'b.')

plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()


plt.semilogy(binned_ell,(fiducial_binned_TT_spectrum -Additive_Bias_TT)*Multiplicative_Bias_est_TT* binned_ell * (binned_ell+1.)/2. / np.pi,'r.')
plt.semilogy(binned_ell,(fiducial_binned_EE_spectrum -Additive_Bias_EE)*Multiplicative_Bias_est_EE* binned_ell * (binned_ell+1.)/2. / np.pi,'g.')
plt.semilogy(binned_ell,(fiducial_binned_BB_spectrum -Additive_Bias_BB)*Multiplicative_Bias_est_BB* binned_ell * (binned_ell+1.)/2. / np.pi,'b.')
plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.semilogy(ell,DlBB,color='y')
plt.semilogy(binned_ell,(Additive_Bias_TT)*Multiplicative_Bias_est_TT* binned_ell * (binned_ell+1.)/2. / np.pi,color='r')
plt.semilogy(binned_ell,(Additive_Bias_EE)*Multiplicative_Bias_est_EE* binned_ell * (binned_ell+1.)/2. / np.pi,color='g')
plt.semilogy(binned_ell,(Additive_Bias_BB)*Multiplicative_Bias_est_BB* binned_ell * (binned_ell+1.)/2. / np.pi,color='b')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()


plt.plot(binned_ell,(fiducial_binned_TE_spectrum- Additive_Bias_TE)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE)* binned_ell * (binned_ell+1.)/2. / np.pi,'r.')
plt.plot(binned_ell,(Additive_Bias_TE)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE)* binned_ell * (binned_ell+1.)/2. / np.pi,'r')


plt.plot(ell,DlTE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,(fiducial_binned_TB_spectrum- Additive_Bias_TB)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'g.')
plt.plot(binned_ell,(fiducial_binned_EB_spectrum- Additive_Bias_EB)*np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'b.')
plt.plot(binned_ell,(Additive_Bias_TB)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'g')
plt.plot(binned_ell,(Additive_Bias_EB)*np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB)* binned_ell * (binned_ell+1.)/2. / np.pi,'b')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()



The green curve in this plot shows our unbiased estimate for the spectrum.  This estimate includes corrections for the noise bias and the transfer function.  The yellow curve shows our estimate for the noise only additive bias.  The blue curve shows the spectrum accounting for only the multiplicative bias as was done above.  The red curve shows the underlying power spectrum used in generating our simulated map.

At this point we have an unbiased estimate of the power spectrum (the green curve). Note that at high-$\ell$  the spectrum is dominated by the SZ and point source components.  Next we need to put error bars on this measurement.  Again we do this with the monty carlo approach.

### quantifying the error bars

The error bars are computed by generating simulations including signal and noise, computing the naive power spectrum, taking the RMS of these results and then subtracting the noise bias and accounting for the transfer function.  In effect we are simulating running the experiment a bunch of times and looking at the RMS of the results--- this approach is broadly applicable to other problems.

In [ ]:
SplusN  = np.zeros([N_spectra_types,N_iterations,int(ell_max/delta_ell)])
i = 0
while (i <N_iterations):
    ## the raw CMB maps
    CMB_T,CMB_Q,CMB_U = make_CMB_maps(N,pix_size,ell,DlTT,DlEE,DlTE,DlBB)
    ## add the point sources
    Source_Amplitude_List=10**(inverse_transform_sampling(hist_pdf, logS_llim, Number_of_Sampled_Sources))
    PSMap_T,PSMap_Q,PSMap_U = generated_source_component(N,pix_size,Number_of_Sampled_Sources,pol_fraction_mean,Source_Amplitude_List,flux_cut) 
    ## add the SZ
    SZMap,trash_catalogue = SZ_source_comonent(N,pix_size,Number_of_SZ_Clusters,Mean_Amplitude_of_SZ_Clusters,SZ_beta,SZ_Theta_core,False)
    ## total the source maps
    CMB_T = CMB_T + PSMap_T + SZMap
    CMB_Q = CMB_Q + PSMap_Q
    CMB_U = CMB_U + PSMap_U
    ## convlove the maps
    CMB_T_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_T)
    CMB_Q_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_Q)
    CMB_U_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_U)
    ## calcualte the noise
    Noise_T = make_noise_map(N,pix_size,white_noise_level,noise_amplitude_1,noise_index_1,noise_amplitude_2,noise_index_2)
    Noise_Q = make_noise_map(N,pix_size,white_noise_level*np.sqrt(2.),noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
    Noise_U = make_noise_map(N,pix_size,white_noise_level*np.sqrt(2.),noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
    ## total maps
    CMB_T_tot = CMB_T_convolved + Noise_T
    CMB_Q_tot = CMB_Q_convolved + Noise_Q
    CMB_U_tot = CMB_U_convolved + Noise_U
    ## write out the maps
    if write_out_maps:
        np.savetxt(filepath + "T"+ str(i) +".dat", CMB_T_tot)
        np.savetxt(filepath + "Q"+ str(i) +".dat", CMB_Q_tot)
        np.savetxt(filepath + "U"+ str(i) +".dat", CMB_U_tot)
    ## calcualte the power spectrum
    fTmap,fEmap,fBmap = kendrick_method_TQU_to_fourier_TEB(N,pix_size,CMB_T_tot,CMB_Q_tot,CMB_U_tot,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)
    
    binned_ell, binned_TT_spectrum = calculate_2d_spectrum(fTmap,fTmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_EE_spectrum = calculate_2d_spectrum(fEmap,fEmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_BB_spectrum = calculate_2d_spectrum(fBmap,fBmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_TE_spectrum = calculate_2d_spectrum(fTmap,fEmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_TB_spectrum = calculate_2d_spectrum(fTmap,fBmap,delta_ell,ell_max,pix_size,N)
    binned_ell, binned_EB_spectrum = calculate_2d_spectrum(fEmap,fBmap,delta_ell,ell_max,pix_size,N)
    SplusN[0,i,:] = binned_TT_spectrum
    SplusN[1,i,:] = binned_EE_spectrum
    SplusN[2,i,:] = binned_BB_spectrum
    SplusN[3,i,:] = binned_TE_spectrum
    SplusN[4,i,:] = binned_TB_spectrum
    SplusN[5,i,:] = binned_EB_spectrum

    if write_out_maps:
        sys.stdout.write("\r signal and noise sims, iterations complete: %d of %d *** writing maps ***" % ((i+1),N_iterations) )
        sys.stdout.flush()
    else:
        sys.stdout.write("\r signal and noise sims, iterations complete: %d of %d" % ((i+1),N_iterations) )
        sys.stdout.flush()
        
    i = i + 1

In [ ]:
mean_sig_plus_noise,rms_sig_plus_noise = average_N_spectra(SplusN,N_iterations,int(ell_max/delta_ell))


Dl_TT_mean = (fiducial_binned_TT_spectrum -Additive_Bias_TT)*Multiplicative_Bias_est_TT * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_TT_std = rms_sig_plus_noise[0,:]*Multiplicative_Bias_est_TT* binned_ell * (binned_ell+1.)/2. / np.pi
Dl_EE_mean = (fiducial_binned_EE_spectrum -Additive_Bias_EE)*Multiplicative_Bias_est_EE * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_EE_std = rms_sig_plus_noise[1,:]*Multiplicative_Bias_est_EE* binned_ell * (binned_ell+1.)/2. / np.pi
Dl_BB_mean = (fiducial_binned_BB_spectrum -Additive_Bias_BB)*Multiplicative_Bias_est_BB * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_BB_std = rms_sig_plus_noise[2,:]*Multiplicative_Bias_est_BB* binned_ell * (binned_ell+1.)/2. / np.pi

Dl_TE_mean = (fiducial_binned_TE_spectrum -Additive_Bias_TE)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_TE_std = rms_sig_plus_noise[3,:]* np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_TB_mean = (fiducial_binned_TB_spectrum -Additive_Bias_TB)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_TB_std = rms_sig_plus_noise[4,:]* np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_EB_mean = (fiducial_binned_EB_spectrum -Additive_Bias_EB)*np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_EB_std = rms_sig_plus_noise[5,:]* np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi



plt.errorbar(binned_ell, Dl_TT_mean,  yerr=Dl_TT_std)
plt.errorbar(binned_ell, Dl_EE_mean,  yerr=Dl_EE_std)
plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.title('high $\ell$ TT and EE')
plt.axis([0, 10000, 1e-3, 1e4])
plt.show()


lowell = np.where(binned_ell < 5000)

plt.errorbar(binned_ell, Dl_TT_mean,  yerr=Dl_TT_std)
plt.errorbar(binned_ell, Dl_EE_mean,  yerr=Dl_EE_std)
plt.errorbar(binned_ell, Dl_BB_mean,  yerr=Dl_BB_std)
plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.semilogy(ell,DlBB,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, 1e-3, 1e4])
plt.show()

plt.errorbar(binned_ell, Dl_TE_mean, yerr=Dl_TE_std)
plt.plot(ell,DlTE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, -150, 150])
plt.show()

plt.errorbar(binned_ell, Dl_TB_mean,  yerr=Dl_TB_std)
plt.errorbar(binned_ell, Dl_EB_mean,  yerr=Dl_EB_std)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, -50, 50])
plt.show()



## beam systematics
here is a beam systematics model.  The point of putting it here is so that we can use the bais and trasnfer functions estiamted above and calculate what the instrumental requiremetns are in the absences of fancy analysis techniques.

In [ ]:
### beam systematics dictionary
bs = {"budy":{"A":1e-3,"FWHP":1.5,"R":14.,"psi":0.3,"polfracQ":0.5,"polfracU":0.01},  
                                                                         # little budy amplitude, 
                                                                         #FWHP, offset spacing, rotationa angle (radians)
                                                                        # pol_fraction for Q and U
     "ghostshelf": {"A":1e-4,"Diam":20.,"roll_off":7.},    #model of ghosting, amplitude (A), diameter (Diam) 
     "hex_crostalk":{"grid_space": 2.5,"N":5,"neighbor_exp_fall":0.01}, 
                                                                               ## model of optical cross talk 
                                                                               ## to detectors on a hex grid
                                                                               ## assumed to be exponetial
                                                                               ## assumed to be 50% polairzed
      "TtoQ":{"mono":1e-3,"dip_x":1e-2,"dip_y":1e-2,"quad_x":1e-2,"quad_45":1e-2},
      "TtoU":{"mono":1e-3,"dip_x":1e-2,"dip_y":1e-2,"quad_x":1e-2,"quad_45":1e-2},
                                                                  ## multiplole expansion leakage
      'psi':0.01*np.pi/180.
                                  ## detetor angle rotations
                    
                   }
    ###############################   
        ############################### 
          ###############################   
              ###############################   
    

def convlolve(A,B): ## the 2d convolution
    fA = np.fft.ifft2(np.fft.fftshift(A))
    fB = np.fft.ifft2(np.fft.fftshift(B))
    convAB = np.fft.fftshift(np.fft.fft2(fA*fB))
    return(np.real(convAB))
  ###############################   
    
def make_2d_beam_cordinates(N,pix_size):
    # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) * pix_size
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2.)
    Theta = np.arctan2(Y,X)
    return(X,Y,R,Theta)
  ###############################   

def Plot_beam(beam_to_Plot,c_min,c_max,N,pix_size,plot_width):
    print("beam max:",np.max(beam_to_Plot),"beam min:",np.std(beam_to_Plot))
    ## set the range
    N_bins = np.floor(plot_width / pix_size / 2.)
    X_width = N_bins * pix_size * 2.
    Y_width = N_bins * pix_size * 2.
    section_to_plot = beam_to_Plot[N/2 - N_bins:N/2 + N_bins,N/2 - N_bins:N/2 + N_bins]
    ##
    plt.figure()
    im = plt.imshow(section_to_plot, interpolation='bilinear', origin='lower',cmap=cm.magma)
    im.set_clim(c_min,c_max)
    cbar = plt.colorbar()
    im.set_extent([0,X_width,0,Y_width])
    plt.ylabel('angle [arcmin]')
    plt.xlabel('angle [arcmin]')
    cbar.set_label('amplitud (arb)', rotation=270)
    plt.show()
    return(0)
  ###############################  

In [ ]:
def make_little_buddies(N,pix_size,beam_size_fwhp,bs,main_beam_peak):
    # make a 2d coordinate system
    X,Y,R,Theta = make_2d_beam_cordinates(N,pix_size)
    #set up four buddies
    budy_sigma = bs["budy"]["FWHP"] / np.sqrt(8.*np.log(2))
    X_rot = X * np.cos(bs["budy"]["psi"]) - Y*np.sin(bs["budy"]["psi"])
    Y_rot = X * np.sin(bs["budy"]["psi"]) + Y*np.cos(bs["budy"]["psi"])
    buddy_beam = 0.
    R_temp = np.sqrt((X_rot - bs["budy"]["R"])**2. + Y_rot**2.)
    buddy_beam+= np.exp(-.5 *(R_temp/budy_sigma)**2.)
    R_temp = np.sqrt((X_rot + bs["budy"]["R"])**2. + Y_rot**2.)
    buddy_beam+= np.exp(-.5 *(R_temp/budy_sigma)**2.)
    R_temp = np.sqrt(X_rot**2. + (Y_rot - bs["budy"]["R"])**2.)
    buddy_beam+= np.exp(-.5 *(R_temp/budy_sigma)**2.)
    R_temp = np.sqrt(X_rot**2. + (Y_rot + bs["budy"]["R"])**2.)
    buddy_beam+= np.exp(-.5 *(R_temp/budy_sigma)**2.)
    buddy_beam = buddy_beam / np.max(buddy_beam)   ##  normalize the budy beam to 1
    ## make the buddy beams
    Budy_TT = buddy_beam / np.max(buddy_beam) * main_beam_peak * bs["budy"]["A"]  ## guarentees the budies are peak normalized realative to main beam
    Budy_QT = buddy_beam / np.max(buddy_beam) * main_beam_peak * bs["budy"]["A"] * bs["budy"]["polfracQ"]
    Budy_UT = buddy_beam / np.max(buddy_beam) * main_beam_peak * bs["budy"]["A"] * bs["budy"]["polfracU"]
    ## return the buddies
    return(Budy_TT,Budy_QT,Budy_UT)
  ############################### 
    
## test the little buddies 
main_beam = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
main_beam_peak = np.max(main_beam)
## budy
Budy_TT,Budy_QT,Budy_UT = make_little_buddies(N,pix_size,beam_size_fwhp,bs,main_beam_peak)

print np.max(Budy_TT), np.max(Budy_QT), np.max(Budy_UT)

beam_to_plot = Budy_TT
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

beam_to_plot = Budy_QT
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

beam_to_plot = Budy_UT
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

In [ ]:
 def make_ghosting_beam(N,pix_size,beam_size_fwhp,bs,main_beam_peak):
    # make a 2d coordinate system
    X,Y,R,Theta = make_2d_beam_cordinates(N,pix_size)
    ## make shelf
    in_shelf = np.where(R < bs["ghostshelf"]["Diam"])
    shelf = np.zeros(np.shape(R))
    shelf[in_shelf] = 1.
    roll_off_kernal = np.exp(-.5 *(R/bs["ghostshelf"]["roll_off"])**2.)
    shelf = convlolve(shelf,roll_off_kernal)
    shelf = shelf / np.max(shelf) * main_beam_peak * bs["ghostshelf"]["A"]  ## normalized relative to the peak of the main beam
    return(shelf)
  ###############################   

## test the ghost
main_beam = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
main_beam_peak = np.max(main_beam)
## ghost
ghosts  = make_ghosting_beam(N,pix_size,beam_size_fwhp,bs,main_beam_peak)

beam_to_plot = ghosts
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

In [ ]:
def make_cross_talk_beam_grid(N,pix_size,beam_size_fwhp,bs):
    # make a 2d coordinate system
    X,Y,R,Theta = make_2d_beam_cordinates(N,pix_size)
    ## make cross-talk grid
    hex_grid = np.zeros(np.shape(R)) 
    delta_Nx = np.floor(bs["hex_crostalk"]["grid_space"] / pix_size)
    delta_Nx_nextrow = delta_Nx /2.
    delta_Ny_nextrow = delta_Nx * np.cos(30. * np.pi/180.)
    i = -1* bs["hex_crostalk"]["N"]
    while (i <= bs["hex_crostalk"]["N"]):
        j = -1* bs["hex_crostalk"]["N"]
        while (j <= bs["hex_crostalk"]["N"]):
            i_active = N/2 + i * delta_Nx
            j_active = N/2 + j * delta_Ny_nextrow
            if (i % 2 == 1):
                j_active -= delta_Nx_nextrow  ## offset every other row
            i_active = int(i_active)
            j_active = int(j_active)
            if ((i_active !=N/2) or (j_active !=N/2)):  # exclude the main beam
                distance = np.sqrt(np.abs((i_active - N/2)/ delta_Nx)**2 + np.abs((j_active - N/2)/delta_Nx)**2)
                xtalk_tau = 1./ np.log(bs["hex_crostalk"]["neighbor_exp_fall"])
                distance /= xtalk_tau
                hex_grid[i_active, j_active] = np.exp(distance)
            j+=1
        i+=1
    const = np.ones(np.shape(hex_grid))
    hex_grid *= np.sum(const)   ## normalize the gain so that the x-talk amplitude is defined correclty
    return(hex_grid)
  ###############################  

main_beam = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
main_beam_peak = np.max(main_beam)

## cross-talk
hex_grid = make_cross_talk_beam_grid(N,pix_size,beam_size_fwhp,bs)
cross_talk = convlolve(hex_grid,main_beam)

beam_to_plot = cross_talk
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

In [ ]:
def make_monopole_dipole_quadrupole(N,pix_size,beam_size_fwhp,bs):
    # make a 2d coordinate system
    X,Y,R,Theta = make_2d_beam_cordinates(N,pix_size)
    # monopole
    beam_sigma = beam_size_fwhp / np.sqrt(8.*np.log(2))
    mono = np.exp(-.5 *(R/beam_sigma)**2.)
    norm = np.sum(mono)
    mono = mono/norm
    # dipoles
    dip_x = mono * R * np.sin(Theta)
    dip_x /= (np.max(dip_x) * norm)
    dip_y = mono * R * np.cos(Theta)
    dip_y /= (np.max(dip_y) * norm)
    # quadrupoles
    quad_x = mono * R * np.sin(2. * Theta)
    quad_x /= (np.max(quad_x) * norm)
    quad_45 = mono * R * np.cos(2. * Theta)
    quad_45 /= (np.max(quad_45) * norm)
    ## return
    return(mono,dip_x,dip_y,quad_x,quad_45)
  ###############################  

#cross_talk = convlolve(hex_grid,main_beam)
## beam systeamtics
mono,dip_x,dip_y,quad_x,quad_45 = make_monopole_dipole_quadrupole(N,pix_size,beam_size_fwhp,bs)


beam_to_plot = mono
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,10)
beam_to_plot = dip_x
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,10)
beam_to_plot = dip_y
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,10)
beam_to_plot = quad_x
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,10)
beam_to_plot = quad_45
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,10)

In [ ]:
## plot the full beam model
## main beam
main_beam = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
main_beam_peak = np.max(main_beam)
## budy
Budy_TT,Budy_QT,Budy_UT = make_little_buddies(N,pix_size,beam_size_fwhp,bs,main_beam_peak)
## ghost
ghosts  = make_ghosting_beam(N,pix_size,beam_size_fwhp,bs,main_beam_peak)
## cross-talk

hex_grid = make_cross_talk_beam_grid(N,pix_size,beam_size_fwhp,bs)
cross_talk = convlolve(hex_grid,main_beam)

#cross_talk = convlolve(hex_grid,main_beam)
## beam systeamtics
mono,dip_x,dip_y,quad_x,quad_45 = make_monopole_dipole_quadrupole(N,pix_size,beam_size_fwhp,bs)


beam_to_plot = cross_talk + main_beam + Budy_TT + ghosts
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

In [ ]:
def make_systematics_beams(N,pix_size,beam_size_fwhp,bs):
    # intitalize the beam to zero
    B_TT=B_QQ=B_UU=B_QT=B_UT=B_QU=B_UQ=0.
    
    # make a 2d gaussian --- this is the main beam 
    main_beam = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
    ## merge this into the beam
    B_TT += main_beam
    B_QQ += main_beam
    B_UU += main_beam
    
    main_beam_peak = np.max(main_beam)
    
    # make the little buddies
    Budy_TT,Budy_QT,Budy_UT = make_little_buddies(N,pix_size,beam_size_fwhp,bs,main_beam_peak)
    ## merge this into the beam
    B_TT += Budy_TT
    B_QT += Budy_QT
    B_UT += Budy_UT

    # make the ghosting shelf
    shelf = make_ghosting_beam(N,pix_size,beam_size_fwhp,bs,main_beam_peak)
    ## merge this into the beam
    B_TT += shelf
    B_QQ += shelf
    B_UU += shelf
    
    # make the cross talk beam
    # make a hex grid centered on the beam
    hex_grid = make_cross_talk_beam_grid(N,pix_size,beam_size_fwhp,bs)
    #convolve the hex grid with the beam
    cross_talk = convlolve(hex_grid,main_beam)
    ## merge this into the beam
    B_TT += cross_talk
    B_QQ += cross_talk
    B_UU += cross_talk
    B_QT += cross_talk
    B_UT += cross_talk
    
    ## add the monopole + dipole + quadrupole T->P leakages
    # make the beam modes
    mono,dip_x,dip_y,quad_x,quad_45 = make_monopole_dipole_quadrupole(N,pix_size,beam_size_fwhp,bs)
    TtoQ = bs["TtoQ"]["mono"] * mono
    TtoQ += bs["TtoQ"]["dip_x"] * dip_x
    TtoQ += bs["TtoQ"]["dip_y"] * dip_y
    TtoQ += bs["TtoQ"]["quad_x"] * quad_x
    TtoQ += bs["TtoQ"]["quad_45"] * quad_45
    TtoU = bs["TtoU"]["mono"] * mono
    TtoU += bs["TtoU"]["dip_x"] * dip_x
    TtoU += bs["TtoU"]["dip_y"] * dip_y
    TtoU += bs["TtoU"]["quad_x"] * quad_x
    TtoU += bs["TtoU"]["quad_45"] * quad_45
    ## add to the beams
    B_QT += TtoQ
    B_UT += TtoU
    
    ## return
    return(B_TT,B_QQ,B_UU,B_QT,B_UT,B_QU,B_UQ)
  ###############################     


####################################################################
####################################################################
####################################################################

B_TT,B_QQ,B_UU,B_QT,B_UT,B_QU,B_UQ = make_systematics_beams(N,pix_size,beam_size_fwhp,bs)

beam_to_plot = B_TT
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

beam_to_plot = B_QQ
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)
                                                            
beam_to_plot = B_UU
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)
                                                            
beam_to_plot = B_QT
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

beam_to_plot = B_UT
beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
beam_to_plot = 10. * np.log10(beam_to_plot)
Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)

#beam_to_plot = B_QU
#beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
##beam_to_plot = 10. * np.log10(beam_to_plot)
#Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)
 
#beam_to_plot = B_UQ
#beam_to_plot = beam_to_plot / np.max(beam_to_plot) + 1e-5
##beam_to_plot = 10. * np.log10(beam_to_plot)
#Plot_beam(beam_to_plot,np.min(beam_to_plot),np.max(beam_to_plot),N,pix_size,50)
 

### polarization angle rotation
this is not stritly a beam systematic, but since it is easy to implemnt, here it is.

In [ ]:
def rotate_poliarzation_vector(Qmap,Umap,bs):
    psi = bs["psi"]
    Qout = Qmap*np.cos(2*psi) - Umap * np.sin(2*psi)
    Uout = Qmap*np.sin(2*psi) - Umap * np.cos(2*psi)    
    return(Qout,Uout)

## next quantify some systmatics

In [ ]:
## generate one realization of the CMB etc, use it for all systematic sims to avoid introducing extra noise
## the raw CMB maps
CMB_T,CMB_Q,CMB_U = make_CMB_maps(N,pix_size,ell,DlTT,DlEE,DlTE,DlBB)
## add the point sources
Source_Amplitude_List=10**(inverse_transform_sampling(hist_pdf, logS_llim, Number_of_Sampled_Sources))
PSMap_T,PSMap_Q,PSMap_U = generated_source_component(N,pix_size,Number_of_Sampled_Sources,pol_fraction_mean,Source_Amplitude_List,flux_cut) 
## add the SZ
SZMap,trash_catalogue = SZ_source_comonent(N,pix_size,Number_of_SZ_Clusters,Mean_Amplitude_of_SZ_Clusters,SZ_beta,SZ_Theta_core,False)
## total the source maps
CMB_T = CMB_T + PSMap_T + SZMap
CMB_Q = CMB_Q + PSMap_Q
CMB_U = CMB_U + PSMap_U
## calcualte a realization of noise
Noise_T = make_noise_map(N,pix_size,white_noise_level,noise_amplitude_1,noise_index_1,noise_amplitude_2,noise_index_2)
Noise_Q = make_noise_map(N,pix_size,white_noise_level*np.sqrt(2.),noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
Noise_U = make_noise_map(N,pix_size,white_noise_level*np.sqrt(2.),noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)

######################################

p=Plot_CMB_Map(CMB_T,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(CMB_Q,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(CMB_U,c_min/20.,c_max/20.,X_width,Y_width)

In [ ]:
## now dress the siulation with systematics 
def convolve_map_polarized_systmatic_beam(B_TT,B_QQ,B_UU,B_QT,B_UT,B_QU,B_UQ,QMap,UMap,TMap):
    "convolves a map with a gaussian beam pattern.  NOTE: pix_size and beam_size_fwhp need to be in the same units" 
    # do the convolutions   
    sys_map_T = convlolve(B_TT,TMap)
    sys_map_Q = convlolve(B_QQ,QMap) + convlolve(B_QT,TMap) #+ convlolve(B_QU,UMap)    
    sys_map_U = convlolve(B_UU,QMap) + convlolve(B_UT,TMap) #+ convlolve(B_UQ,QMap) 
    # return the convolved map
    return(sys_map_T,sys_map_Q,sys_map_U)
  ###############################   
    
## now dress the siulation with systematics
## convlove the maps to add systeamtics
B_TT,B_QQ,B_UU,B_QT,B_UT,B_QU,B_UQ = make_systematics_beams(N,pix_size,beam_size_fwhp,bs)
## rotate the overall polarizaiton angle
CMB_Q,CMB_U = rotate_poliarzation_vector(CMB_Q,CMB_U,bs)
## convolve to add beam systeatmics
CMB_T_syst,CMB_Q_syst,CMB_U_syst = convolve_map_polarized_systmatic_beam(B_TT,B_QQ,B_UU,B_QT,B_UT,B_QU,B_UQ,CMB_Q,CMB_U,CMB_T)



## total maps
CMB_T_tot_sys = CMB_T_syst #+ Noise_T
CMB_Q_tot_sys = CMB_Q_syst #+ Noise_Q
CMB_U_tot_sys = CMB_U_syst #+ Noise_U

p=Plot_CMB_Map(CMB_T,c_min,c_max,X_width,Y_width)
p=Plot_CMB_Map(CMB_T_tot_sys,c_min,c_max/10.,X_width,Y_width)

p=Plot_CMB_Map(CMB_Q,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(CMB_Q_tot_sys,c_min/20.,c_max/20.,X_width,Y_width)

p=Plot_CMB_Map(CMB_U,c_min/20.,c_max/20.,X_width,Y_width)
p=Plot_CMB_Map(CMB_U_tot_sys,c_min/20.,c_max/20.,X_width,Y_width)






In [ ]:
## calculate the power spectrum

## make a power spectrum
fTmap_sys,fEmap_sys,fBmap_sys = kendrick_method_TQU_to_fourier_TEB(N,pix_size,CMB_T_tot_sys,CMB_Q_tot_sys,CMB_U_tot_sys,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)
binned_ell, binned_TT_spectrum_sys = calculate_2d_spectrum(fTmap_sys,fTmap_sys,delta_ell,ell_max,pix_size,N)
binned_ell, binned_EE_spectrum_sys = calculate_2d_spectrum(fEmap_sys,fEmap_sys,delta_ell,ell_max,pix_size,N)
binned_ell, binned_BB_spectrum_sys = calculate_2d_spectrum(fBmap_sys,fBmap_sys,delta_ell,ell_max,pix_size,N)
binned_ell, binned_TE_spectrum_sys = calculate_2d_spectrum(fTmap_sys,fEmap_sys,delta_ell,ell_max,pix_size,N)
binned_ell, binned_TB_spectrum_sys = calculate_2d_spectrum(fTmap_sys,fBmap_sys,delta_ell,ell_max,pix_size,N)
binned_ell, binned_EB_spectrum_sys = calculate_2d_spectrum(fEmap_sys,fBmap_sys,delta_ell,ell_max,pix_size,N)



Dl_TT_mean_sys = (binned_TT_spectrum_sys -Additive_Bias_TT)*Multiplicative_Bias_est_TT * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_EE_mean_sys = (binned_EE_spectrum_sys -Additive_Bias_EE)*Multiplicative_Bias_est_EE * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_BB_mean_sys = (binned_BB_spectrum_sys -Additive_Bias_BB)*Multiplicative_Bias_est_BB * binned_ell * (binned_ell+1.)/2. / np.pi

Dl_TE_mean_sys = (binned_TE_spectrum_sys -Additive_Bias_TE)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_TB_mean_sys = (binned_TB_spectrum_sys -Additive_Bias_TB)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
Dl_EB_mean_sys = (fiducial_binned_EB_spectrum -Additive_Bias_EB)*np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi


## look at deviations from the zero systeamtics analysis
# make plots

plt.errorbar(binned_ell, Dl_TT_mean,  yerr=Dl_TT_std)
plt.errorbar(binned_ell, Dl_EE_mean,  yerr=Dl_EE_std)
plt.errorbar(binned_ell, Dl_TT_mean_sys,  yerr=Dl_TT_std)
plt.errorbar(binned_ell, Dl_EE_mean_sys,  yerr=Dl_EE_std)

plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.title('high $\ell$ TT and EE')
plt.axis([0, 10000, 1e-3, 1e4])
plt.show()


lowell = np.where(binned_ell < 5000)

plt.errorbar(binned_ell, Dl_TT_mean,  yerr=Dl_TT_std)
plt.errorbar(binned_ell, Dl_EE_mean,  yerr=Dl_EE_std)
plt.errorbar(binned_ell, Dl_BB_mean,  yerr=Dl_BB_std)
plt.errorbar(binned_ell, Dl_TT_mean_sys,  yerr=Dl_TT_std)
plt.errorbar(binned_ell, Dl_EE_mean_sys,  yerr=Dl_EE_std)
plt.errorbar(binned_ell, Dl_BB_mean_sys,  yerr=Dl_BB_std)

plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.semilogy(ell,DlBB,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, 1e-3, 1e4])
plt.show()

plt.errorbar(binned_ell, Dl_TE_mean, yerr=Dl_TE_std)
plt.errorbar(binned_ell, Dl_TE_mean_sys, yerr=Dl_TE_std)
plt.plot(ell,DlTE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, -150, 150])
plt.show()

plt.errorbar(binned_ell, Dl_TB_mean,  yerr=Dl_TB_std)
plt.errorbar(binned_ell, Dl_EB_mean,  yerr=Dl_EB_std)
plt.errorbar(binned_ell, Dl_TB_mean_sys,  yerr=Dl_TB_std)
plt.errorbar(binned_ell, Dl_EB_mean_sys,  yerr=Dl_EB_std)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, -50, 50])
plt.show()


# calcualte chi_squared

chi_sq_TT = np.sum((Dl_TT_mean - Dl_TT_mean_sys)**2 / Dl_TT_std**2)
chi_sq_EE = np.sum((Dl_EE_mean - Dl_EE_mean_sys)**2 / Dl_EE_std**2)
chi_sq_BB = np.sum((Dl_BB_mean - Dl_BB_mean_sys)**2 / Dl_BB_std**2)

chi_sq_TE = np.sum((Dl_TE_mean - Dl_TE_mean_sys)**2 / Dl_TE_std**2)
chi_sq_TB = np.sum((Dl_TB_mean - Dl_TB_mean_sys)**2 / Dl_TB_std**2)
chi_sq_EB = np.sum((Dl_EB_mean - Dl_EB_mean_sys)**2 / Dl_EB_std**2)

print "chi_sq: TT: ",chi_sq_TT,"EE:",chi_sq_EE,"BB: ",chi_sq_BB
print "chi_sq: TE: ",chi_sq_TT,"TB:",chi_sq_TB,"EB: ",chi_sq_EB

And there you have it.  That is how you compute a CMB power spectrum and error bars.   If you want to fit cosmology to these data you can re-run CAMB varying cosmological parameters and compute the likelihood difference between these models and the data.  This is left as an exercise.

## SZ and point source analysis techniques

A few point sources and SZ sources are visible in our simulated map, however there are many that are faint and difficult to identify.  There are sophisticated techniques for isolating these sources.  We now discuss the matched filter approach.

###Constructing a matcher filter

We construct a single frequency matched filter following Melin etc...  the punch line is that we build the following filter in fourier space:

$$\psi = \frac{B_\ell S_\ell }{B_\ell^2 N^2_{ap,\ell} +N^2_{ins,\ell} } $$

Both $\psi$ and all the other quantities are two dimensional in the since that they depend on $k_x$ and $k_y$.  We refer to these coordinates with a $\ell$ as shorthand.   In this equation $B_\ell$ is the Fourier transform of the beam pattern and filtering; $S_\ell$ is the Fourier transform of the signal template; $N^2_{ap,\ell}$ is the power spectrum (e.g., absolute value squared of the 2d FFT) of the astrophysical noise from sources other than what is desired to be found, and $N^2_{ins,\ell}$ is the power spectrum of the instrumental response.

Following what we did above to compute the power spectrum, we will calculate the denominator of the filter using monty carlo techniques.  We then plot the raw map, the matched filtered map, and a histogram of the pixels in the matched filter.

Once the filter is computed it can be applied by multiplying the map by the filter in Fourier space.  The filtered map is recovered with a simple inverse Fourier transform.

In [ ]:
def matched_filter(input_map,beam_and_filt,cluster_profile,FT_noise_covar):
    FT_beam_and_filt = np.fft.fft2(np.fft.fftshift(beam_and_filt))
    FT_signal = np.fft.fft2(np.fft.fftshift(cluster_profile))
    
    psi = FT_beam_and_filt * FT_signal / FT_noise_covar
    
    filtered = psi * np.fft.fft2(np.fft.fftshift(input_map))
    filtered = np.fft.fftshift(np.fft.ifft2(filtered))
    filtered = np.real(filtered)
    return(filtered)

                                
## construct the beam and cluster profile
beam_and_filt = make_2d_gaussian_beam(N,pix_size,beam_size_fwhp)
cluster_profile = beta_function(N,pix_size,SZ_beta,SZ_Theta_core)
pt_source_profile = np.zeros([N,N])
pt_source_profile[N/2,N/2] = 1.

## construct the 2d noise power spectrum
FT_noise_covar = np.zeros((N,N))
window = (cosine_window(N))
i = 0
while (i <N_iterations):
     ## the raw CMB maps
    CMB_T,CMB_Q,CMB_U = make_CMB_maps(N,pix_size,ell,DlTT,DlEE,DlTE,DlBB)
    ## add the point sources
    PSMap = Poisson_source_comonent(N,pix_size,Number_of_Sources,Amplitude_of_Sources) 
    PSMap +=  Exponential_source_comonent(N,pix_size,Number_of_Sources_EX,Amplitude_of_Sources_EX)
    SZMap,trash = SZ_source_comonent(N,pix_size,Number_of_SZ_Clusters,Mean_Amplitude_of_SZ_Clusters,SZ_beta,SZ_Theta_core,False)
    ## total the source maps
    CMB_T  = CMB_T  + PSMap + SZMap
    ## convlove the maps
    CMB_T_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,CMB_T)
    #CMB_Q_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,Q_total_map)
    #CMB_U_convolved = convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,U_total_map)
    ## calcualte the noise
    Noise_T = make_noise_map(N,pix_size,white_noise_level,noise_amplitude_1,noise_index_1,noise_amplitude_2,noise_index_2)
    #Noise_Q = make_noise_map(N,pix_size,white_noise_level_pol,noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
    #Noise_U = make_noise_map(N,pix_size,white_noise_level_pol,noise_amplitude_1_pol,noise_index_1,noise_amplitude_2_pol,noise_index_2)
    ## total maps
    CMB_T_tot = CMB_T_convolved + Noise_T
    #CMB_Q_tot = CMB_Q_convolved + Noise_Q
    #CMB_U_tot = CMB_U_convolved + Noise_U
    ## calcualte the power spectrum
    
    temp =  np.fft.fft2(np.fft.fftshift(window* (CMB_T_tot)))
    FT_noise_covar += np.real(np.conj(temp)*temp/(N_iterations*1.0))
    sys.stdout.write("\r matched filter noise realization, iterations complete: %d of %d" % ((i+1),N_iterations) )
    sys.stdout.flush()
    i = i + 1

In [ ]:
def Plot_Matched_Filtered_Map(Map_to_Plot,X_width,Y_width,c_min,c_max):
    print("map mean:",np.mean(Map_to_Plot),"map rms:",np.std(Map_to_Plot))
    plt.figure()
    im = plt.imshow(Map_to_Plot, interpolation='bilinear', origin='lower',cmap=cm.RdBu_r)
    im.set_clim(c_min,c_max)
    cbar = plt.colorbar()
    im.set_extent([0,X_width,0,Y_width])
    plt.ylabel('angle $[^\circ]$')
    plt.xlabel('angle $[^\circ]$')
    cbar.set_label('matched_filter [S/N]', rotation=270)
    plt.show()
    return(0)
  ###############################

def RMS_with_CUT(MAP,threshold,N_iter):
    RMS = np.std(MAP)
    AVG = np.mean(MAP)
    i = 0
    while (i < N_iter):
        chi_sq = np.abs((MAP - AVG) / RMS)
        ok = np.where(chi_sq < threshold)
        RMS = np.std(MAP[ok])
        AVG = np.mean(MAP[ok])
        i = i + 1
        print RMS,AVG
    return(RMS)
  ###############################
    
    
pt_source_profile = np.zeros([N,N])
pt_source_profile[N/2,N/2] = 1.
    
## make the filtered maps
SZ_filtered_map = matched_filter(T_total_map_plus_noise,beam_and_filt,cluster_profile,FT_noise_covar)
pt_source_filtered_map = matched_filter(T_total_map_plus_noise,beam_and_filt,pt_source_profile,FT_noise_covar)

## make a S/N map
SZ_SN_map = SZ_filtered_map / RMS_with_CUT(SZ_filtered_map,7,5)  ## 7 sigma cut, 5 iterations
PS_SN_map = pt_source_filtered_map / RMS_with_CUT(pt_source_filtered_map,7,5)

## make a few plots
p = Plot_CMB_Map(T_total_map_plus_noise,c_min,c_max,X_width,Y_width)
p = Plot_Matched_Filtered_Map(SZ_SN_map,X_width,Y_width,-10,20)
p = Plot_Matched_Filtered_Map(PS_SN_map,X_width,Y_width,-10,20)

####BP = plot_3maps_bokeh(T_total_map_plus_noise,SZ_SN_map,PS_SN_map) ### Warning--- cool looking plot, but precludes saving
#p = Plot_Matched_Filtered_Map(PS_SN_map-SZ_SN_map,X_width,Y_width,-3,3)

hist_SZ,bin_edges_SZ = np.histogram(SZ_SN_map,bins = 100,range=[SZ_SN_map.min(),SZ_SN_map.max()])
hist_PS,bin_edges_PS = np.histogram(PS_SN_map,bins = 100,range=[PS_SN_map.min(),PS_SN_map.max()])

plt.semilogy(bin_edges_SZ[0:-1],hist_SZ,'r')
plt.semilogy(bin_edges_PS[0:-1],hist_PS,'b')
plt.title('red: SZ filter, blue: PS filter')
plt.ylabel('number of pixels')
plt.xlabel('matched filter signal to noise')
plt.show()

In [ ]:
k_max_filter = 2500.  ## high pass filter for plotting maps with high pass


def HighPass_Map(N,pix_size,K_min,Map):
    ## construct 2-d k-coordinates
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) /(N-1.)
    kX = np.outer(ones,inds) / (pix_size/60. * np.pi/180.)
    kY = np.transpose(kX)
    K = np.sqrt(kX**2. + kY**2.)
    ell_scale_factor = 2. * np.pi 
    ell2d = K * ell_scale_factor
    ## make a mask
    mask = np.ones([N,N])
    bad = np.where(ell2d < K_min)
    mask[bad] = 0.
    ## apply the mask
    FT_Map = np.fft.fft2(Map)
    mask =   np.fft.fftshift(mask)
    ## inverse transform
    highpassed_map = np.real(np.fft.ifft2(mask*FT_Map))
    ## return
    return(highpassed_map)
    ###############################    

    
## apply a high pass filter

#SZ_filtered_map = HighPass_Map(N,pix_size,k_max_filter,SZ_filtered_map)
#pt_source_filtered_map = HighPass_Map(N,pix_size,k_max_filter,pt_source_filtered_map)
T_map_highpass = HighPass_Map(N,pix_size,k_max_filter,T_total_map_plus_noise)
Q_map_highpass = HighPass_Map(N,pix_size,k_max_filter,Q_total_map_plus_noise)
U_map_highpass = HighPass_Map(N,pix_size,k_max_filter,U_total_map_plus_noise)
SZ_SN_map_highpass = HighPass_Map(N,pix_size,k_max_filter,SZ_SN_map)
PS_SN_map_highpass = HighPass_Map(N,pix_size,k_max_filter,PS_SN_map)




p = Plot_Matched_Filtered_Map(SZ_SN_map_highpass,X_width,Y_width,-10,20)
p = Plot_Matched_Filtered_Map(PS_SN_map_highpass,X_width,Y_width,-10,20)

p = Plot_CMB_Map(T_map_highpass,c_min,c_max,X_width,Y_width)
p = Plot_CMB_Map(Q_map_highpass,c_min,c_max,X_width,Y_width)
p = Plot_CMB_Map(U_map_highpass,c_min,c_max,X_width,Y_width)


### extracting a cluster list from the matched filtered maps

In [ ]:
SN_min = 4.0
Radius_to_Avoid  = 25  ## number of pixels to avoid at the edges of the map

def extract_catalogue_sourcesubtract(filtered_map,templet,SN_min,Radius_to_Avoid):
    Extracted_Cat = [[-1,-1,-1,-1]]
    maxpix = filtered_map.max()
    i = 0  ## an index to count the number of objects found
    while (SN_min < maxpix):
        masked_map = filtered_map
        masked_map[0:Radius_to_Avoid,:] =0
        masked_map[-Radius_to_Avoid:,:] =0
        masked_map[:,0:Radius_to_Avoid] =0
        masked_map[:,-Radius_to_Avoid:] =0
        
        maxpix = masked_map.max()
        pos_max_pix = np.where(masked_map == maxpix)
        xc = pos_max_pix[0]
        xc = xc[0]
        yc = pos_max_pix[1]
        yc = yc[0]
        
        stamp = filtered_map[xc-Radius_to_Avoid:xc+Radius_to_Avoid,yc-Radius_to_Avoid:yc+Radius_to_Avoid]
        

        filtered_map =  filtered_map - maxpix * np.roll(np.roll(templet,xc,axis = 0),yc,axis = 1)
        cleaned_stamp = filtered_map[xc-Radius_to_Avoid:xc+Radius_to_Avoid,yc-Radius_to_Avoid:yc+Radius_to_Avoid]
        templet_stamp = (np.roll(np.roll(templet,xc,axis = 0),yc,axis = 1))[xc-Radius_to_Avoid:xc+Radius_to_Avoid,yc-Radius_to_Avoid:yc+Radius_to_Avoid]
       
        ## fill in the catalogue
        Extracted_Cat_new = [[i,xc,yc,maxpix]]
        Extracted_Cat = np.append(Extracted_Cat,Extracted_Cat_new,axis = 0)
        
        ## make informative plots
        print Extracted_Cat_new
        p = Plot_CMB_Map(stamp,maxpix * -1.,maxpix,Radius_to_Avoid*2,Radius_to_Avoid*2)
        p = Plot_CMB_Map(templet_stamp,maxpix * -1.,maxpix,Radius_to_Avoid*2,Radius_to_Avoid*2)
        p = Plot_CMB_Map(cleaned_stamp,maxpix * -1,maxpix,Radius_to_Avoid*2,Radius_to_Avoid*2)
        print '--------------------'
        
        ## increment the object counter
        i = i + 1
        
    return(Extracted_Cat[1:-1,:])
  ###############################
    
def extract_catalogue_mask(filtered_map,mask_templet,SN_min,Radius_to_Avoid):
    Extracted_Cat = [[-1,-1,-1,-1]]
    maxpix = filtered_map.max()
    i = 0  ## an index to count the number of objects found
    while (SN_min < maxpix):
        masked_map = filtered_map
        masked_map[0:Radius_to_Avoid,:] =0
        masked_map[-Radius_to_Avoid:,:] =0
        masked_map[:,0:Radius_to_Avoid] =0
        masked_map[:,-Radius_to_Avoid:] =0
        
        maxpix = masked_map.max()
        pos_max_pix = np.where(masked_map == maxpix)
        xc = pos_max_pix[0]
        xc = xc[0]
        yc = pos_max_pix[1]
        yc = yc[0]
        
        stamp = filtered_map[xc-Radius_to_Avoid:xc+Radius_to_Avoid,yc-Radius_to_Avoid:yc+Radius_to_Avoid]
        
        mask_templet_toapply = np.roll(np.roll(mask_templet,xc,axis = 0),yc,axis = 1)
        
        filtered_map =  filtered_map *mask_templet_toapply
        cleaned_stamp = filtered_map[xc-Radius_to_Avoid:xc+Radius_to_Avoid,yc-Radius_to_Avoid:yc+Radius_to_Avoid]
       
        ## fill in the catalogue
        Extracted_Cat_new = [[i,xc,yc,maxpix]]
        Extracted_Cat = np.append(Extracted_Cat,Extracted_Cat_new,axis = 0)
        
        ## make informative plots
        print Extracted_Cat_new
        p = Plot_CMB_Map(stamp,maxpix * -1.,maxpix,Radius_to_Avoid*2,Radius_to_Avoid*2)
        p = Plot_CMB_Map(cleaned_stamp,maxpix * -1,maxpix,Radius_to_Avoid*2,Radius_to_Avoid*2)
        print '--------------------'
        
        ## increment the object counter
        i = i + 1
        
    return(Extracted_Cat[1:-1,:])
  ###############################




cluster_profile = beta_function(N,pix_size,SZ_beta,SZ_Theta_core)

cluster_profile = cluster_profile + np.roll(cluster_profile,1,axis = 1)
cluster_profile = cluster_profile + np.roll(cluster_profile,1,axis = 0)
cluster_profile = cluster_profile * 0.25


    
ConvSZBeam = 2.5 * np.fft.fftshift(convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,cluster_profile))
#recovered_SZ_catalaouge = extract_catalogue_sourcesubtract(SZ_SN_map*(-1.),ConvSZBeam*(1.),SN_min,Radius_to_Avoid)


mask_templet = np.ones([N,N])
cluster_pix = np.where(ConvSZBeam > 0.15)
mask_templet[cluster_pix] = 0.
recovered_SZ_catalaouge = extract_catalogue_mask(SZ_SN_map*(-1.),mask_templet,SN_min,Radius_to_Avoid)

In [ ]:
print recovered_SZ_catalaouge

### extract the point sources

In [ ]:
pt_source_profile = np.zeros([N,N])
pt_source_profile[N/2,N/2] = 0.25
pt_source_profile[N/2,N/2+1] = 0.25
pt_source_profile[N/2+1,N/2] = 0.25
pt_source_profile[N/2+1,N/2+1] = 0.25

    
ConvPSBeam = 11. * np.fft.fftshift(convolve_map_with_gaussian_beam(N,pix_size,beam_size_fwhp,pt_source_profile))

#recovered_PS_catalaouge = extract_catalogue(PS_SN_map,ConvPSBeam,SN_min,Radius_to_Avoid)

mask_templet = np.ones([N,N])
cluster_pix = np.where(ConvPSBeam > 0.15)
mask_templet[cluster_pix] = 0.
recovered_PS_catalaouge = extract_catalogue_mask(PS_SN_map,mask_templet,SN_min,Radius_to_Avoid)

In [ ]:
print recovered_PS_catalaouge

#Analyzing the ACTPol Maps

In [ ]:
Nx = 7080.
Ny = 1818.
X_width_full = Nx*pix_size/60.  # horizontal map width in degrees
Y_width_full = Ny*pix_size/60.  # vertical map width in degrees

### read in the maps
dataT_D5_6_56=pyfits.getdata(ACTMapfilepath + 'D5+6+56_not_converged_maps/deep5+6+56_night_tot_map0100_T_sM.fits')   #Tmap
dataQ_D5_6_56=pyfits.getdata(ACTMapfilepath + 'D5+6+56_not_converged_maps/deep5+6+56_night_tot_map0100_Q_sM.fits')     #Qmap 
dataU_D5_6_56=pyfits.getdata(ACTMapfilepath + 'D5+6+56_not_converged_maps/deep5+6+56_night_tot_map0100_U_sM.fits')  #Umap
ACTPol_T_D5_6_56=dataT_D5_6_56#[0:1818,512:2330]
ACTPol_Q_D5_6_56=dataQ_D5_6_56#[0:1818,512:2330]
ACTPol_U_D5_6_56=dataU_D5_6_56#[0:1818,512:2330]
print np.shape(ACTPol_T_D5_6_56)
### read in a source list
src_list_D5_6_56 = np.loadtxt(ACTMapfilepath + 'D5+6+56_not_converged_maps/catalog5_6_56_pix.txt')   # select source list
#pixX = src_list_D5_6_56[:,0]     #RA
#pixY = src_list_D5_6_56[:,1]     #Dec 
#value_of_detection = src_list_D5_6_56[:,2]  #in uK
#SNR = src_list_D5_6_56[:,3]   #SNR


## give all the data products generic names so the code below always works
src_list = src_list_D5_6_56
ACTPol_T = ACTPol_T_D5_6_56 * 1.0
ACTPol_Q = ACTPol_Q_D5_6_56 * 1.0
ACTPol_U = ACTPol_U_D5_6_56 * 1.0 * (-1.)

ACTPol_T_full = ACTPol_T
ACTPol_Q_full = ACTPol_Q
ACTPol_U_full = ACTPol_U

## the limits xs,xe and ys, ye are defined above in the noise modeling section

ACTPol_T = ACTPol_T[xs:xe,ys:ye]
ACTPol_Q = ACTPol_Q[xs:xe,ys:ye]
ACTPol_U = ACTPol_U[xs:xe,ys:ye]

## plot the ACT map
p = Plot_CMB_Map(ACTPol_T,c_min,c_max,X_width,Y_width)
p = Plot_CMB_Map(ACTPol_Q,c_min,c_max,X_width,Y_width)
p = Plot_CMB_Map(ACTPol_U,c_min,c_max,X_width,Y_width)

## print the source list
print np.shape(src_list)


Filter the ACTPol Maps

In [ ]:
## plot filtered versions of the ACTPol maps
print  np.shape(ACTPol_T)
filter_scale = 5.  ## arcminutes
k_max_filter = 300.
ACTPol_T_filtered = convolve_map_with_gaussian_beam(N,pix_size,filter_scale,ACTPol_T)
ACTPol_Q_filtered = convolve_map_with_gaussian_beam(N,pix_size,filter_scale,ACTPol_Q)
ACTPol_U_filtered = convolve_map_with_gaussian_beam(N,pix_size,filter_scale,ACTPol_U)

ACTPol_T_filtered = HighPass_Map(N,pix_size,k_max_filter,ACTPol_T_filtered)
ACTPol_Q_filtered = HighPass_Map(N,pix_size,k_max_filter,ACTPol_Q_filtered)
ACTPol_U_filtered = HighPass_Map(N,pix_size,k_max_filter,ACTPol_U_filtered)

p = Plot_CMB_Map(ACTPol_T_filtered,c_min,c_max,X_width,Y_width)
p = Plot_CMB_Map(ACTPol_Q_filtered,c_min/20.,c_max/20.,X_width,Y_width)
p = Plot_CMB_Map(ACTPol_U_filtered,c_min/20.,c_max/20.,X_width,Y_width)

In [ ]:
## make a nieve power spectrum
ACTPol_fTmap,ACTPol_fEmap,ACTPol_fBmap = kendrick_method_TQU_to_fourier_TEB(N,pix_size,ACTPol_T,ACTPol_Q,ACTPol_U,window,dwin_dx,dwin_dy,d2win_dx2,d2win_dy2,d2win_dxdy)
binned_ell, ACTPol_binned_TT_spectrum = calculate_2d_spectrum(ACTPol_fTmap,ACTPol_fTmap,delta_ell,ell_max,pix_size,N)
binned_ell, ACTPol_binned_EE_spectrum = calculate_2d_spectrum(ACTPol_fEmap,ACTPol_fEmap,delta_ell,ell_max,pix_size,N)
binned_ell, ACTPol_binned_BB_spectrum = calculate_2d_spectrum(ACTPol_fBmap,ACTPol_fBmap,delta_ell,ell_max,pix_size,N)
binned_ell, ACTPol_binned_TE_spectrum = calculate_2d_spectrum(ACTPol_fTmap,ACTPol_fEmap,delta_ell,ell_max,pix_size,N)
binned_ell, ACTPol_binned_TB_spectrum = calculate_2d_spectrum(ACTPol_fTmap,ACTPol_fBmap,delta_ell,ell_max,pix_size,N)
binned_ell, ACTPol_binned_EB_spectrum = calculate_2d_spectrum(ACTPol_fEmap,ACTPol_fBmap,delta_ell,ell_max,pix_size,N)


## plot the ACTPol maps
plt.semilogy(binned_ell,ACTPol_binned_TT_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(binned_ell,ACTPol_binned_EE_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(binned_ell,ACTPol_binned_BB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
## plot the simulated maps
plt.semilogy(binned_ell,fiducial_binned_TT_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(binned_ell,fiducial_binned_EE_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(binned_ell,fiducial_binned_BB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.semilogy(ell,DlTT)
#plt.semilogy(ell,DlEE)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,ACTPol_binned_TE_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(binned_ell,fiducial_binned_TE_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(ell,DlTE)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

plt.plot(binned_ell,ACTPol_binned_TB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(binned_ell,ACTPol_binned_EB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(binned_ell,fiducial_binned_TB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.plot(binned_ell,fiducial_binned_EB_spectrum* binned_ell * (binned_ell+1.)/2. / np.pi)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.show()

In [ ]:
## make the corrected ACTPol spectra
ACTPol_Dl_TT_mean = (ACTPol_binned_TT_spectrum -Additive_Bias_TT)*Multiplicative_Bias_est_TT * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_TT_std = rms_sig_plus_noise[0,:]*Multiplicative_Bias_est_TT* binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_EE_mean = (ACTPol_binned_EE_spectrum -Additive_Bias_EE)*Multiplicative_Bias_est_EE * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_EE_std = rms_sig_plus_noise[1,:]*Multiplicative_Bias_est_EE* binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_BB_mean = (ACTPol_binned_BB_spectrum -Additive_Bias_BB)*Multiplicative_Bias_est_BB * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_BB_std = rms_sig_plus_noise[2,:]*Multiplicative_Bias_est_BB* binned_ell * (binned_ell+1.)/2. / np.pi

ACTPol_Dl_TE_mean = (ACTPol_binned_TE_spectrum -Additive_Bias_TE)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE) * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_TE_std = rms_sig_plus_noise[3,:]* np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_EE) * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_TB_mean = (ACTPol_binned_TB_spectrum -Additive_Bias_TB)*np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_TB_std = rms_sig_plus_noise[4,:]* np.sqrt(Multiplicative_Bias_est_TT * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_EB_mean = (ACTPol_binned_EB_spectrum -Additive_Bias_EB)*np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi
ACTPol_Dl_EB_std = rms_sig_plus_noise[5,:]* np.sqrt(Multiplicative_Bias_est_EE * Multiplicative_Bias_est_BB) * binned_ell * (binned_ell+1.)/2. / np.pi

plt.errorbar(binned_ell, ACTPol_Dl_TT_mean,  yerr=ACTPol_Dl_TT_std)
plt.errorbar(binned_ell, ACTPol_Dl_EE_mean,  yerr=ACTPol_Dl_EE_std)
#plt.errorbar(binned_ell, Dl_TT_mean,  yerr=Dl_TT_std)
#plt.errorbar(binned_ell, Dl_EE_mean,  yerr=Dl_EE_std)
plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.title('high $\ell$ TT and EE')
plt.axis([0, 10000, 1e-3, 1e4])
plt.show()


lowell = np.where(binned_ell < 5000)

plt.errorbar(binned_ell, ACTPol_Dl_TT_mean,  yerr=ACTPol_Dl_TT_std)
plt.errorbar(binned_ell, ACTPol_Dl_EE_mean,  yerr=ACTPol_Dl_EE_std)
plt.errorbar(binned_ell, ACTPol_Dl_BB_mean,  yerr=ACTPol_Dl_BB_std)
#plt.errorbar(binned_ell, Dl_TT_mean,  yerr=Dl_TT_std)
#plt.errorbar(binned_ell, Dl_EE_mean,  yerr=Dl_EE_std)
#plt.errorbar(binned_ell, Dl_BB_mean,  yerr=Dl_BB_std)
plt.semilogy(ell,DlTT,color='y')
plt.semilogy(ell,DlEE,color='y')
plt.semilogy(ell,DlBB,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, 1e-3, 1e4])
plt.show()


plt.errorbar(binned_ell, ACTPol_Dl_TE_mean, yerr=ACTPol_Dl_TE_std)
#plt.errorbar(binned_ell, Dl_TE_mean, yerr=Dl_TE_std)
plt.plot(ell,DlTE,color='y')
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, -150, 150])
plt.show()

plt.errorbar(binned_ell, ACTPol_Dl_TB_mean,  yerr=ACTPol_Dl_TB_std)
plt.errorbar(binned_ell, ACTPol_Dl_EB_mean,  yerr=ACTPol_Dl_EB_std)
#plt.errorbar(binned_ell, Dl_TB_mean,  yerr=Dl_TB_std)
#plt.errorbar(binned_ell, Dl_EB_mean,  yerr=Dl_EB_std)
plt.ylabel('$D_{\ell}$ [$\mu$K$^2$]')
plt.xlabel('$\ell$')
plt.axis([0, 5000, -50, 50])
plt.show()

In [ ]:
## make the filtered maps
ACTPol_SZ_filtered_map = matched_filter(ACTPol_T,beam_and_filt,cluster_profile,FT_noise_covar)
ACTPol_pt_source_filtered_map = matched_filter(ACTPol_T,beam_and_filt,pt_source_profile,FT_noise_covar)

## make a S/N map
ACTPol_SZ_SN_map = ACTPol_SZ_filtered_map / RMS_with_CUT(ACTPol_SZ_filtered_map,7,5)  ## 7 sigma cut, 5 iterations
ACTPol_PS_SN_map = ACTPol_pt_source_filtered_map / RMS_with_CUT(ACTPol_pt_source_filtered_map,7,5) 


## make a few plots
p = Plot_CMB_Map(ACTPol_T,c_min,c_max,X_width,Y_width)
p = Plot_Matched_Filtered_Map(ACTPol_SZ_SN_map,X_width,Y_width,-10,20)
p = Plot_Matched_Filtered_Map(ACTPol_PS_SN_map,X_width,Y_width,-10,20)
#p = Plot_Matched_Filtered_Map(PS_SN_map-SZ_SN_map,X_width,Y_width,-3,3)

hist_SZ,bin_edges_SZ = np.histogram(ACTPol_SZ_SN_map,bins = 100,range=[SZ_SN_map.min(),ACTPol_SZ_SN_map.max()])
hist_PS,bin_edges_PS = np.histogram(ACTPol_PS_SN_map,bins = 100,range=[PS_SN_map.min(),ACTPol_PS_SN_map.max()])

plt.semilogy(bin_edges_SZ[0:-1],hist_SZ,'r')
plt.semilogy(bin_edges_PS[0:-1],hist_PS,'b')
plt.title('red: SZ filter, blue: PS filter')
plt.ylabel('number of pixels')
plt.xlabel('matched filter signal to noise')
plt.show()

In [ ]:
pt_source_profile = np.zeros([N,N])
pt_source_profile[N/2,N/2] = 0.25
pt_source_profile[N/2,N/2+1] = 0.25
pt_source_profile[N/2+1,N/2] = 0.25
pt_source_profile[N/2+1,N/2+1] = 0.25

    
ConvPSBeam = 12.2 * np.fft.fftshift(convolve_map_with_ACTPol_beam(N,pix_size,ACTPol_beam_file,pt_source_profile))

#recovered_PS_catalaouge = extract_catalogue(ACTPol_PS_SN_map,ConvPSBeam,SN_min,Radius_to_Avoid)

mask_templet = np.ones([N,N])
cluster_pix = np.where(ConvPSBeam > 0.01)
mask_templet[cluster_pix] = 0.
recovered_PS_catalaouge = extract_catalogue_mask(ACTPol_PS_SN_map,mask_templet,SN_min,Radius_to_Avoid)


In [ ]:
print recovered_PS_catalaouge

Here we show three maps: (top) the raw simulated map, (middle) the matched filtered map, and (bottom) a histogram of the S/N of each pixel in the matched filter map.  You could identify SZ sources by doing a S/N cut and taking all the pixels below say -5.  Next you would throw out neighbors and that would give you an SZ source catalogue.   To find bright poit sources we would replace the beta function in the definition of the filter with the beam and otherwise repeat the exercise looking for S/N greater than say 5.  Since a 1 arc minute beta function is similar to the beam the results would not change substantially.

There you have it--- the matched filter makes many sources visible, and the histogram reveals that there are both a large number of SZ and point source like objects in the matched filtered map.

## Stacking

One often wants to understand signals at the low mass and therefore low signal to noise end of things.   If an external catalogue exists, one can add the signals from objects in bins to boost the signal to noise.   In this example we stack the observed map and the matched filtered map at the positions of SZ clusters.  This highlights the opportunities that exist with stacking.  Interpreting these results requires careful analysis.  Monte carlo simulations including selection effects and systematic effects are a particularly appealing approach.  This should be apparent at this point.  Though many details must be added to this code to realize this goal.  We will provide a simple example (centering errors) below.

In [ ]:
## take SZCat and stack total_map_plus_noise on the SZ positions, do this in a mass bin

cat = src_list
N_objects = src_list.shape[0]
bin_min = 5.  # Only 2 sources in the chosen ~70 square degrees with SNR>100
bin_max = 50.
Radius_stacking = 6*2.

def Stack_on_Positions(map,Nx,Ny,cat,N_objects,bin_min,bin_max,Radius):
    stack = np.zeros([Radius*2,Radius*2])
    counter = 0
    i = 0
    while (i < N_objects):
        ampl = cat[i,3]
        if ((ampl > bin_min) and (ampl <= bin_max)):
            yc = cat[i,0] +1.
            xc = cat[i,1] +1.
            if ((xc > Radius) and (xc < Ny-Radius)):
                if ((yc > Radius) and (yc < Nx-Radius)):
                    #print xc, yc
                    #print i,counter,cat[i,:]
                    stack += map[xc-Radius:xc+Radius,yc-Radius:yc+Radius]
                    #print np.max(stack)
                    counter +=1.0
        i = i + 1
    print counter
    return(stack/counter)

print Nx, Ny
#stack_SNT = Stack_on_Positions(ACTPol_PS_SN_map,Nx,Ny,cat,N_objects,bin_min,bin_max,Radius_stacking)
stack_T = Stack_on_Positions(ACTPol_T_full,Nx,Ny,cat,N_objects,bin_min,bin_max,Radius_stacking)
stack_Q = Stack_on_Positions(ACTPol_Q_full,Nx,Ny,cat,N_objects,bin_min,bin_max,Radius_stacking)
stack_Q = stack_Q - np.mean(stack_Q)
stack_U = Stack_on_Positions(ACTPol_U_full,Nx,Ny,cat,N_objects,bin_min,bin_max,Radius_stacking)
stack_U = stack_U - np.mean(stack_U)


print "pol fractions: ", np.array([np.sum(stack_Q),np.sum(stack_U)]) / np.sum(stack_T) 

#p = Plot_CMB_Map(stack_SNT,c_min,c_max,X_width*50*2/Nx,Y_width*50*2/Ny)
p = Plot_CMB_Map(stack_T,np.min(stack_T),np.max(stack_T),Radius_stacking,Radius_stacking)
p = Plot_CMB_Map(stack_Q,np.min(stack_Q),np.max(stack_Q),Radius_stacking,Radius_stacking)
p = Plot_CMB_Map(stack_U,np.min(stack_U),np.max(stack_U),Radius_stacking,Radius_stacking)




***********

***********

***********

***********

***********

***********

##this is as far as I have gotten in implementing polariation

***********

***********

***********

***********

***********

***********

The upper plot shows the CMB temperature map stacked on the locations of SZ clusters.  The lower plot shows the same stacking exercise repeated with the matched filtered map.

### centering errors

In this example we introduce errors in the positions of the clusters in the catalogue.  It is obvious that the cluster signal broadens and washes out. 

In [ ]:

centering_errors_x = np.random.normal(0,2,Number_of_SZ_Clusters)
centering_errors_y = np.random.normal(0,2,Number_of_SZ_Clusters)
SZCat_centering_errs = SZCat
SZCat_centering_errs[0,:]  += centering_errors_x
SZCat_centering_errs[1,:]  += centering_errors_y

stack = Stack_on_Positions(total_map_plus_noise,N,SZCat_centering_errs,Number_of_SZ_Clusters,-100000,100000,50)
p = Plot_CMB_Map(stack,c_min/4.,c_max/4.,X_width*50*2/N,Y_width*50*2/N)





Compare this plot with the perfectly centered map to understand what centering errors do to the stacking.

In [ ]:
def numerical_gradient(N,pix_size,beam_size_fwhp,seperation_angular_size,Map):
    "gaussian smoothed numerical gradient" 
    # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) * pix_size
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2.)
  
    # make a 2d gaussian 
    beam_sigma = beam_size_fwhp / np.sqrt(8.*np.log(2))
    gaussian = np.exp(-.5 *(R/beam_sigma)**2.)
    gaussian = gaussian / np.sum(gaussian)
    
    # shift the gaussians and make one negative
    shift_bins = int(seperation_angular_size / pix_size/2.)
    kern_x = np.roll(gaussian,shift_bins,0) - np.roll(gaussian,-1* shift_bins,0)
    kern_y = np.roll(gaussian,shift_bins,1) - np.roll(gaussian,-1* shift_bins,1)
    
    # do the convolution
    FT_kern_x = np.fft.fft2(np.fft.fftshift(kern_x))
    FT_kern_y = np.fft.fft2(np.fft.fftshift(kern_y))
    FT_Map = np.fft.fft2(np.fft.fftshift(Map))
    gradient_map = (np.fft.fftshift(np.real(np.fft.ifft2(FT_kern_x*FT_Map))))**2.
    gradient_map += (np.fft.fftshift(np.real(np.fft.ifft2(FT_kern_y*FT_Map))))**2.
    gradient_map = np.sqrt(gradient_map)
    # return the convolved map
    return(gradient_map)
  ###############################    

## plot the numerical gradient

gradientMap = numerical_gradient(N,pix_size,1.5,3,total_map_plus_noise)
plt.figure()
im = plt.imshow(gradientMap, interpolation='bilinear', origin='lower',cmap=cm.RdBu)
plt.show()




## CMB Lensing


In [ ]:
# read in the input lensing spectra
ell, ltothe4Clphi = np.loadtxt("/Users/jeffmcm/Desktop/CAMB_fiducial_cosmo_scalCls.dat", usecols=(0, 4), unpack=True) 
ltothe4Clphi = ltothe4Clphi/ 7.4311e12  # propertly divide out an arbitray scaling as described in the CAMB readme
Cl_phiphi = ltothe4Clphi / (ell**4.)  ## lensing potential power spectrum
plt.loglog(ell,Cl_phiphi)
plt.ylabel('$\ell^4 C_\ell^\phi$')
plt.xlabel('$\ell$')
plt.show()

In [ ]:
def make_CMB_Deflection_map(N,pix_size,ell,Cl_phiphi):
    "makes a realization of CMB temperature anisotropies"

    # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) /(N-1.)
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2. + Y**2.)
    
    # now make a 2d lensing mask in k/ell space
    ell_scale_factor = 2. * np.pi / (pix_size/60. * np.pi/180.)
    ell2d = R * ell_scale_factor
    Cl_phiphi_expanded = np.zeros(ell2d.max()+1)
    Cl_phiphi_expanded[0:(Cldd.size)] = Cl_phiphi
    CCl_phiphi2d = Cl_phiphi_expanded[ell2d.astype(int)]
 
    # now make the lensing maps (for x and y directions)
    ramdomn_array_for_deflections = np.fft.fft2(np.random.normal(0,1,(N,N)))
    FT_2d = np.sqrt(CCl_phiphi2d) * ramdomn_array_for_deflections
    CMB_phimap = np.fft.ifft2(np.fft.fftshift(FT_2d)) /(pix_size /60.* np.pi/180.)
    CMB_phimap = np.real(CMB_phimap)
    
    CMB_deflection_x = (np.roll(CMB_phimap,-4, axis =0) - np.roll(CMB_phimap,4, axis =0) ) /(8. * pix_size/60. *np.pi/180.)
    CMB_deflection_y = (np.roll(CMB_phimap,-4, axis =1) - np.roll(CMB_phimap,4, axis =1) ) /(8. * pix_size/60. *np.pi/180.) 

    ## return the map
    return(CMB_deflection_x,CMB_deflection_y)
  ###############################

def Plot_deflection_Map(Map_to_Plot,c_min,c_max,X_width,Y_width):
    print("map mean:",np.mean(Map_to_Plot),"map rms:",np.std(Map_to_Plot))
    plt.figure()
    im = plt.imshow(Map_to_Plot, interpolation='bilinear', origin='lower',cmap=cm.RdBu_r)
    im.set_clim(c_min,c_max)
    cbar = plt.colorbar()
    im.set_extent([0,X_width,0,Y_width])
    plt.ylabel('angle $[^\circ]$')
    plt.xlabel('angle $[^\circ]$')
    cbar.set_label('deflection angle [arcmin]', rotation=270)
    plt.show()
    return(0)
  ###############################

## make a CMB T map
CMB_deflection_x,CMB_deflection_y = make_CMB_Deflection_map(N,pix_size,ell,Cldd)
CMB_deflection_x *= 60. * 180./np.pi
CMB_deflection_y *= 60. * 180./np.pi

c = np.std(CMB_deflection_x)*2
p = Plot_deflection_Map((CMB_deflection_x),-c,c,X_width,Y_width)
p = Plot_deflection_Map((CMB_deflection_y),-c,c,X_width,Y_width)

In [ ]:
def apply_lensing(sourcemap,x_deflections,y_deflections):
    '''
    take the source map and the lensing deflection maps, apply the lensing deflections to the
    source map using the openCV cv2 remap function
    return the lensed output

    deflections are given in pixel values
    '''
    xsize, ysize = shape(sourcemap)
    xvals, yvals = meshgrid(range(xsize), range(ysize))
    xnew = float32(xvals - x_deflections)
    ynew = float32(yvals - y_deflections)

    lensed_map = cv2.remap(sourcemap, xnew, ynew, interpolation = cv2.INTER_AREA)
    #check that the sourcemap and the deflection maps are all the same size
    

    return lensed_map